In [ ]:
import data_utils
import math
import numpy as np
import os


def augment_vertices_scale(vertices):
  scale_normalizing_factors = np.random.uniform(low=0.75, high=1.25, size=(3,))
  # min + (max - min) * scale_normalizing_factors (normalizing_factors)
  scale_normalizing_factors = np.reshape(scale_normalizing_factors, [1, 3])
  minn_verts = np.min(vertices, axis=0, keepdims=True)
  vertices = minn_verts + (vertices - minn_verts) * scale_normalizing_factors
  # vertices = vertices * scale_normalizing_factors
  return vertices


# a two part version
# warping_coordinates
# warp coord...
def warping_coordinates(sv_dataset_root_folder, dataset_root_folder, n_scale_sample=1, part_names=[], indices_list=None, motion_cat="eyeglasses"):

  first_obj_fn = os.path.join(dataset_root_folder, "0001")
  obj_fns = os.listdir(first_obj_fn)
  obj_fns = [fn for fn in obj_fns if fn.endswith(".obj")]
  part_names = [fn.split(".")[0] for fn in obj_fns]

  for part_nm in part_names:
    cur_part_sv_root_folder = os.path.join(sv_dataset_root_folder, part_nm)
    os.makedirs(cur_part_sv_root_folder, exist_ok=True)
  
  ins_list = os.listdir(dataset_root_folder)
  ins_list = [fn for fn in ins_list if os.path.isdir(os.path.join(dataset_root_folder, fn))]
  for cur_ins_fn in ins_list:
    tot_cur_ins_fn = os.path.join(dataset_root_folder, cur_ins_fn)
    cur_ins_part_fns = os.listdir(tot_cur_ins_fn)
    cur_ins_part_fns = [fn for fn in cur_ins_part_fns if fn.endswith(".obj")]
    cur_ins_part_fns = [fn.split(".")[0] for fn in cur_ins_part_fns]
    if motion_cat in ["eyeglasses"]:
      cur_ins_part_fns = [fn for fn in cur_ins_part_fns if fn != "none_motion"]
    for cur_ins_part_nm in cur_ins_part_fns:
      cur_part_sv_root_folder = os.path.join(sv_dataset_root_folder, cur_ins_part_nm)
      if not os.path.exists(cur_part_sv_root_folder):
        os.makedirs(cur_part_sv_root_folder, exist_ok=True)
    for part_nm in cur_ins_part_fns:
      cur_ins_cur_part_fn = os.path.join(tot_cur_ins_fn, f"{part_nm}.obj")
      for i_scale_sample in range(n_scale_sample):
        cur_ins_cur_part_sv_fn = os.path.join(sv_dataset_root_folder, part_nm, f"{cur_ins_fn}_scale_{i_scale_sample}.obj")
        cur_part_verts, cur_part_faces = data_utils.read_obj_file_ours(cur_ins_cur_part_fn)

        if not (cur_part_verts.shape[0] > 0 and len(cur_part_faces) > 0):
          continue

        verts_min, verts_max = cur_part_verts.min(0), cur_part_verts.max(0) # 3 3
        cur_part_verts = data_utils.center_vertices(cur_part_verts)

        normalized_verts = data_utils.normalize_vertices_scale(cur_part_verts) # n x 3 normalized vertices
        normalized_verts = normalized_verts + 0.5

        normalized_verts = augment_vertices_scale(normalized_verts)

        sigma = 0.25
        x_warp_gradient_sampled = np.random.normal(loc=0.0, scale=math.sqrt(sigma), size=(5,))
        y_warp_gradient_sampled = np.random.normal(loc=0.0, scale=math.sqrt(sigma), size=(5,))
        z_warp_gradient_sampled = np.random.normal(loc=0.0, scale=math.sqrt(sigma), size=(5,))

        x_warp_gradient_sampled = np.exp(x_warp_gradient_sampled)
        y_warp_gradient_sampled = np.exp(y_warp_gradient_sampled)
        z_warp_gradient_sampled = np.exp(z_warp_gradient_sampled)

        # x_reflect = True
        x_reflect = False
        y_reflect = True
        z_reflect = True

        scaled_verts = []
        for i_v in range(normalized_verts.shape[0]):
          cur_v_xyz = normalized_verts[i_v].tolist()
          cur_x, cur_y, cur_z = float(cur_v_xyz[0]), float(cur_v_xyz[1]), float(cur_v_xyz[2])
          scaled_x = data_utils.warp_coord(x_warp_gradient_sampled, cur_x, reflect=x_reflect)
          scaled_y = data_utils.warp_coord(y_warp_gradient_sampled, cur_y, reflect=y_reflect)
          scaled_z = data_utils.warp_coord(z_warp_gradient_sampled, cur_z, reflect=z_reflect)
          scaled_verts.append([scaled_x, scaled_y, scaled_z])
        scaled_verts = np.array(scaled_verts, dtype=np.float) # n x 3
        scaled_verts = data_utils.center_vertices(scaled_verts)
        # normalize_vertices_scale --
        scaled_verts = data_utils.normalize_vertices_scale(scaled_verts)
        data_utils.save_obj_file(scaled_verts, cur_part_faces, cur_ins_cur_part_sv_fn)



sv_dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_scaled"
os.makedirs(sv_dataset_root_folder, exist_ok=True)
# sv_dataset_root_folder = os.path.join(sv_dataset_root_folder, "eyeglasses")
# os.makedirs(sv_dataset_root_folder, exist_ok=True)

dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci"
# dataset_root_folder = os.path.join(dataset_root_folder, "eyeglasses")

# tot_motion_cats = os.listdir(dataset_root_folder)
# tot_motion_cats = [fnn for fnn in tot_motion_cats if os.path.isdir(os.path.join(cat_folder, fnn)) and fnn != "eyeglasses"]

# for r
tot_motion_cats = ["water_bottle", "door"]

tot_motion_cats = ["eyeglasses"]

tot_motion_cats = ["lamp", "screwdriver", "seesaw", "pen"]

for motion_cat in tot_motion_cats:
  cur_sv_dataset_root_folder = os.path.join(sv_dataset_root_folder, motion_cat)
  cur_dataset_root_folder = os.path.join(dataset_root_folder, motion_cat)
  warping_coordinates(cur_sv_dataset_root_folder, cur_dataset_root_folder, n_scale_sample=100, part_names=["none_motion", "dof_rootd_Aa001_r.obj"], motion_cat=motion_cat)

In [35]:


import data_utils
import math
import numpy as np
import os


def augment_vertices_scale(vertices):
  scale_normalizing_factors = np.random.uniform(low=0.75, high=1.25, size=(3,))
  # min + (max - min) * scale_normalizing_factors (normalizing_factors)
  scale_normalizing_factors = np.reshape(scale_normalizing_factors, [1, 3])
  minn_verts = np.min(vertices, axis=0, keepdims=True)
  vertices = minn_verts + (vertices - minn_verts) * scale_normalizing_factors
  # vertices = vertices * scale_normalizing_factors
  return vertices


# a two part version
# warping_coordinates
# warp coord...
def merge_data_part_first(sv_dataset_root_folder, dataset_root_folder, n_scale_sample=1, part_names=[], indices_list=None, motion_cat="eyeglasses"):

  first_obj_fn = os.path.join(dataset_root_folder, "0001")
  obj_fns = os.listdir(first_obj_fn)
  obj_fns = [fn for fn in obj_fns if fn.endswith(".obj")]
  part_names = [fn.split(".")[0] for fn in obj_fns]

  for part_nm in part_names:
    cur_part_sv_root_folder = os.path.join(sv_dataset_root_folder, part_nm)
    os.makedirs(cur_part_sv_root_folder, exist_ok=True)
  
  ins_list = os.listdir(dataset_root_folder)
  ins_list = [fn for fn in ins_list if os.path.isdir(os.path.join(dataset_root_folder, fn))]
  for cur_ins_fn in ins_list:
    tot_cur_ins_fn = os.path.join(dataset_root_folder, cur_ins_fn)
    cur_ins_part_fns = os.listdir(tot_cur_ins_fn)
    cur_ins_part_fns = [fn for fn in cur_ins_part_fns if fn.endswith(".obj")]
    cur_ins_part_fns = [fn.split(".")[0] for fn in cur_ins_part_fns]
    # if motion_cat in ["eyeglasses"]:
    #   # cur_ins_part_fns = [fn for fn in cur_ins_part_fns if fn != "none_motion"]
    #   cur_ins_part_fns = ["none_motion"]
    for cur_ins_part_nm in cur_ins_part_fns:
      cur_part_sv_root_folder = os.path.join(sv_dataset_root_folder, cur_ins_part_nm)
      if not os.path.exists(cur_part_sv_root_folder):
        os.makedirs(cur_part_sv_root_folder, exist_ok=True)
    for part_nm in cur_ins_part_fns:
      cur_ins_cur_part_fn = os.path.join(tot_cur_ins_fn, f"{part_nm}.obj")
      # for i_scale_sample in range(n_scale_sample):
      cur_ins_cur_part_sv_fn = os.path.join(sv_dataset_root_folder, part_nm, f"{cur_ins_fn}.obj")
      cur_part_verts, cur_part_faces = data_utils.read_obj_file_ours(cur_ins_cur_part_fn)

      data_utils.save_obj_file(cur_part_verts, cur_part_faces, cur_ins_cur_part_sv_fn)



# sv_dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_50"
sv_dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_part_first"
os.makedirs(sv_dataset_root_folder, exist_ok=True)
# sv_dataset_root_folder = os.path.join(sv_dataset_root_folder, "eyeglasses")
# os.makedirs(sv_dataset_root_folder, exist_ok=True)

# dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_50"
dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized"
# dataset_root_folder = os.path.join(dataset_root_folder, "eyeglasses")

tot_motion_cats = os.listdir(dataset_root_folder)
tot_motion_cats = [fnn for fnn in tot_motion_cats if os.path.isdir(os.path.join(dataset_root_folder, fnn))]

tot_motion_cats_sv = os.listdir(sv_dataset_root_folder)
tot_motion_cats_sv = [fnn for fnn in tot_motion_cats_sv if os.path.isdir(os.path.join(sv_dataset_root_folder, fnn))]

# tot_motion_c
tot_motion_cats = [fn for fn in tot_motion_cats if fn not in tot_motion_cats_sv]

# # for r
# tot_motion_cats = ["water_bottle", "door"]

# tot_motion_cats = ["eyeglasses"]

# tot_motion_cats = ["lamp", "screwdriver", "seesaw", "pen"]

for motion_cat in tot_motion_cats: # tot_motion_cats
  print(f"processing category: {motion_cat}")
  cur_sv_dataset_root_folder = os.path.join(sv_dataset_root_folder, motion_cat)
  cur_dataset_root_folder = os.path.join(dataset_root_folder, motion_cat)
  merge_data_part_first(cur_sv_dataset_root_folder, cur_dataset_root_folder, n_scale_sample=100, part_names=["none_motion", "dof_rootd_Aa001_r.obj"], motion_cat=motion_cat)

processing category: globe
processing category: screwdriver
processing category: bike
processing category: wine_bottle
processing category: watch
processing category: rocking_chair
processing category: stapler
processing category: lamp
processing category: excavator
processing category: car
processing category: lighter
processing category: tank
processing category: seesaw
processing category: bucket
processing category: fan
processing category: closestool
processing category: washing_machine
processing category: oven
processing category: faucet
processing category: cabinet
processing category: scissors
processing category: door
processing category: pen
processing category: water_bottle
processing category: eyeglasses
processing category: clock


In [ ]:



def get_pts_numbers(datasest_root_folder):
  samples_list = os.listdir(datasest_root_folder)
  samples_list = [fn for fn in samples_list if os.path.isdir(os.path.join(dataset_root_folder, fn))]
  summary_obj_fn = "none_motion.obj"
  for i_s, sample_nm in enumerate(samples_list):
    
    cur_sample_folder = os.path.join(dataset_root_folder, sample_nm)
    cur_sample_summary_obj = os.path.join(cur_sample_folder, summary_obj_fn)
    cur_sample_vertices, cur_sample_faces = data_utils.read_obj(cur_sample_summary_obj)

    nn_face_indices = sum([len(sub_face) for sub_face in cur_sample_faces])

    # if not (cur_sample_vertices.shape[0] <= nn_max_permite_vertices and nn_face_indices <= nn_max_permite_faces):
    #   continue
    
    ### centralize vertices ###
    ins_vert_min = cur_sample_vertices.min(axis=0) # vert_min
    ins_vert_max = cur_sample_vertices.max(axis=0) # vert_max
    ins_vert_center = 0.5 * (ins_vert_min + ins_vert_max)
    cur_sample_vertices = cur_sample_vertices - ins_vert_center
    ins_vert_min = cur_sample_vertices.min(axis=0) # vert_min
    ins_vert_max = cur_sample_vertices.max(axis=0) # vert_max
    ins_extents = ins_vert_max - ins_vert_min # extents
    ins_scale = np.sqrt(np.sum(ins_extents**2)) # scale # 
    cur_sample_vertices = cur_sample_vertices / ins_scale


    cur_sample_mesh_dict = data_utils.process_mesh(cur_sample_vertices, cur_sample_faces, quantization_bits=8, recenter_mesh=True)

    mesh_dict = {}
    mesh_dict = cur_sample_mesh_dict
    mesh_dict['class_label'] = 0
    vertices, faces = mesh_dict['vertices'], mesh_dict['faces']
    print(f'sample_nm: { sample_nm}, nn_vertices: {vertices.shape}, nn_faces: {len(faces)}')

    # mesh_dicts.append(mesh_dict)



dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci"
dataset_root_folder = os.path.join(dataset_root_folder, "eyeglasses")
get_pts_numbers(dataset_root_folder)
# 1, 2, 3, 8, 23, 13, 6, 21, 17, 9, 15, 5, 4, 14, 12, 10, 11, 16, 7, 17, 19, 18



In [2]:
import data_utils

def preprocess_shapenet_data(cat_folder, sv_cat_folder):
  ins_folders = os.listdir(cat_folder)
  # 
  ins_folders = [fn for fn in ins_folders if os.path.isdir(os.path.join(cat_folder, fn))]
  for ins in ins_folders:
    cur_ins_sv_folder = os.path.join(sv_cat_folder, ins)
    os.makedirs(cur_ins_sv_folder, exist_ok=True)
    cur_ins_folder = os.path.join(cat_folder, ins, "part_objs")
    
    cur_ins_part_obj_fns = os.listdir(cur_ins_folder)

    for cur_part_obj_fn in cur_ins_part_obj_fns:
      if not cur_part_obj_fn.endswith(".obj"):
        continue
      tot_cur_part_obj_fn = os.path.join(cur_ins_folder, cur_part_obj_fn)
      cur_ins_verts, cur_ins_faces = data_utils.read_obj_file_ours(tot_cur_part_obj_fn)
      
      tot_cur_part_obj_sv_fn = os.path.join(cur_ins_sv_folder, cur_part_obj_fn)
      data_utils.save_obj_file(cur_ins_verts, cur_ins_faces, tot_cur_part_obj_sv_fn)


dataset_root = "/nas/datasets/gen/datasets"
motion_category = "eyeglasses"
cat_folder = "MotionDataset"
cat_folder = os.path.join(dataset_root, cat_folder)
sv_cat_folder = "MotionDataset_processed"
sv_cat_folder = os.path.join(dataset_root, sv_cat_folder)
os.makedirs(sv_cat_folder, exist_ok=True)

tot_motion_cats_processed = os.listdir(sv_cat_folder)
tot_motion_cats_processed = [fnn for fnn in tot_motion_cats_processed if os.path.isdir(os.path.join(sv_cat_folder, fnn))]


tot_motion_cats = os.listdir(cat_folder)
# tot_motion_cats = [fnn for fnn in tot_motion_cats if os.path.isdir(os.path.join(cat_folder, fnn)) and fnn != "eyeglasses"]
tot_motion_cats = [fnn for fnn in tot_motion_cats if os.path.isdir(os.path.join(cat_folder, fnn))]

tot_motion_cats = [fnn for fnn in tot_motion_cats if fnn not in tot_motion_cats_processed]

# tot_motion_cats = ["lamp", "screwdriver", "seesaw", "pen"]


for motion_category in tot_motion_cats:
  print(f"Preprocessing category {motion_category}...")
  cur_cat_folder = os.path.join(cat_folder, motion_category)
  cur_sv_cat_folder = os.path.join(sv_cat_folder, motion_category)
  os.makedirs(cur_sv_cat_folder, exist_ok=True)
  preprocess_shapenet_data(cur_cat_folder, cur_sv_cat_folder)

Preprocessing category valve...
Preprocessing category handcart...
Preprocessing category window...
Preprocessing category refrigerator...
Preprocessing category swiss_army_knife...
Preprocessing category kettle...
Preprocessing category plane...
Preprocessing category motorbike...
Preprocessing category folding_chair...
Preprocessing category cannon...
Preprocessing category skateboard...
Preprocessing category revolver...
Preprocessing category swing...
Preprocessing category swivel_chair...
Preprocessing category carton...
Preprocessing category helicopter...
Preprocessing category windmill...


In [1]:

import data_utils

def preprocess_shapenet_data(cat_folder, sv_cat_folder):
  ins_folders = os.listdir(cat_folder)
  ins_folders = [fn for fn in ins_folders if os.path.isdir(os.path.join(cat_folder, fn))]
  for ins in ins_folders:
    cur_ins_sv_folder = os.path.join(sv_cat_folder, ins)
    # os.makedirs(cur_ins_sv_folder, exist_ok=True)
    cur_ins_folder = os.path.join(cat_folder, ins)
    
    cur_ins_part_obj_fns = os.listdir(cur_ins_folder)

    for cur_part_obj_fn in cur_ins_part_obj_fns:
      if not cur_part_obj_fn.endswith(".obj"):
        continue
      tot_cur_part_obj_fn = os.path.join(cur_ins_folder, cur_part_obj_fn)
      cur_ins_verts, cur_ins_faces = data_utils.read_obj_file_ours(tot_cur_part_obj_fn) # read obj file ours
      
      tot_cur_part_obj_sv_fn = os.path.join(cur_ins_sv_folder, cur_part_obj_fn)
      data_utils.save_obj_file(cur_ins_verts, cur_ins_faces, tot_cur_part_obj_sv_fn)

motion_category = "eyeglasses"
# cat_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_50/"
# sv_cat_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_50/"
# os.makedirs(sv_cat_folder, exist_ok=True)

cat_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_0.2/"
sv_cat_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_0.2/"



# sv_cat_folder = os.path.join(sv_cat_folder, motion_category)
# os.makedirs(sv_cat_folder, exist_ok=True)

tot_motion_cats = os.listdir(cat_folder)
# tot_motion_cats = [fnn for fnn in tot_motion_cats if os.path.isdir(os.path.join(cat_folder, fnn)) and fnn != "eyeglasses"]
tot_motion_cats = [fnn for fnn in tot_motion_cats if os.path.isdir(os.path.join(cat_folder, fnn))]

# tot_motion_cats = ["eyeglasses"]
# not_tot
# _motion_cats = ["lamp", "screwdriver", "seesaw", "pen", "eyeglasses", "oven"]
# tot_motion_cats = [fnn for fnn in tot_motion_cats if fnn not in not_tot_motion_cats]



for motion_cat in tot_motion_cats:
  print(f"Processing category {motion_cat}...")
  cur_cat_folder = os.path.join(cat_folder, motion_cat)
  cur_sv_cat_folder = os.path.join(sv_cat_folder, motion_cat)
  preprocess_shapenet_data(cur_cat_folder, cur_sv_cat_folder)

Processing category globe...
Processing category screwdriver...
Processing category bike...
Processing category wine_bottle...
Processing category watch...
Processing category rocking_chair...
Processing category stapler...
Processing category lamp...
Processing category excavator...
Processing category car...
Processing category lighter...
Processing category tank...
Processing category seesaw...
Processing category bucket...
Processing category fan...
Processing category closestool...
Processing category washing_machine...
Processing category oven...
Processing category faucet...
Processing category cabinet...
Processing category scissors...
Processing category door...
Processing category pen...
Processing category water_bottle...
Processing category eyeglasses...
Processing category clock...


In [ ]:
import data_utils
import numpy as np

# sv_dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_scaled"
# os.makedirs(sv_dataset_root_folder, exist_ok=True)

# sv_dataset_root_folder = os.path.join(sv_dataset_root_folder, "eyeglasses")
# os.makedirs(sv_dataset_root_folder, exist_ok=True)

# dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci"
dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_scaled"
# dataset_root_folder = os.path.join(dataset_root_folder, "eyeglasses")

tot_motion_cats = os.listdir(dataset_root_folder)
# tot_motion_cats = [fnn for fnn in tot_motion_cats if os.path.isdir(os.path.join(cat_folder, fnn)) and fnn != "eyeglasses"]

tot_motion_cats = ["oven"]

# fo
nn_max_permite_vertices = 400
nn_max_permite_faces = 2000
for motion_cat in tot_motion_cats:
  cur_cat_root_folder = os.path.join(dataset_root_folder, motion_cat)
  cur_cat_part_nms = os.listdir(cur_cat_root_folder)
  cur_cat_part_nms = [fn for fn in cur_cat_part_nms if os.path.isdir(os.path.join(cur_cat_root_folder, fn))]
  
  valid_fns = []
  for part_nm in cur_cat_part_nms:
    cur_cat_cur_part_folder = os.path.join(cur_cat_root_folder, part_nm)
    cur_cat_cur_part_objs = os.listdir(cur_cat_cur_part_folder)
    cur_cat_cur_part_objs = [fn for fn in cur_cat_cur_part_objs if fn.endswith(".obj")]
    print(f"cur motion cat: {motion_cat}, part name: {part_nm}")
    for cur_obj in cur_cat_cur_part_objs:
      cur_obj_tot_fn = os.path.join(cur_cat_cur_part_folder, cur_obj)
      cur_sample_vertices, cur_sample_faces = data_utils.read_obj(cur_obj_tot_fn)
      cur_sample_mesh_dict = data_utils.process_mesh(cur_sample_vertices, cur_sample_faces, quantization_bits=8, recenter_mesh=True)
      vertices, faces = cur_sample_mesh_dict['vertices'], cur_sample_mesh_dict['faces']

      nn_vertices = vertices.shape[0]
      nn_faces = faces.shape[0]
      if not (nn_vertices <= nn_max_permite_vertices and nn_faces <= nn_max_permite_faces): # load and test whether they...
        continue
      else:
        valid_fns.append(cur_obj)
    valid_fns_sv_fn = f"nn_max_verts_{nn_max_permite_vertices}_faces_{nn_max_permite_faces}_valid_fns.npy"
    valid_fns_sv_fn = os.path.join(cur_cat_cur_part_folder, valid_fns_sv_fn)
    np.save(valid_fns_sv_fn, valid_fns)
    


In [ ]:


import os

def get_file_scales(data_folder):
  objs_tot_folders = os.listdir(data_folder)
  print(f"total objs: {objs_tot_folders}")
  objs_tot_folders = [fn for fn in objs_tot_folders if os.path.isdir(os.path.join(data_folder, fn))]
  print(f"after folder filtering: {len(objs_tot_folders)}")

data_folder = "/nas/datasets/gen/datasets/PartNet/data_v0/" # partnet and data...
get_file_scales(data_folder=data_folder)
# we can load data on the fly if we have the knowledge of {ins_idx: valid_obj_names}

In [ ]:


def get_processed_data_stats(data_root_folder, motion_cats):
  cat_to_stats = {}
  for i_cat, cur_cat in enumerate(motion_cats):
    print(f"Cateogory: {cur_cat}")
    cur_cat_data_folder = os.path.join(data_root_folder, cur_cat)
    cur_cat_part_names = os.listdir(cur_cat_data_folder)
    cur_cat_part_names = [fn for fn in cur_cat_part_names if os.path.isdir(os.path.join(cur_cat_data_folder, fn))]
    cur_part_to_stats = {}
    for cur_cat_part in cur_cat_part_names:
      print(f" Part: {cur_cat_part}")
      cur_part_nn_verts, cur_part_nn_face_indices = [], []
      cur_cat_part_folder = os.path.join(cur_cat_data_folder, cur_cat_part)
      cur_cat_part_objs = os.listdir(cur_cat_part_folder)
      cur_cat_part_objs = [fn for fn in cur_cat_part_objs if fn.endswith(".obj")]
      ins_to_exist = {}
      for cur_obj in cur_cat_part_objs:
        ins_idx = cur_obj.split("_")[0]
        if ins_idx in ins_to_exist:
          continue
        ins_to_exist[ins_idx] = 1
        cur_obj_tot_fn = os.path.join(cur_cat_part_folder, cur_obj)
        cur_verts, cur_faces = data_utils.read_obj_file_ours(cur_obj_tot_fn)
        cur_nn_verts = cur_verts.shape[0]
        cur_nn_faces = sum([len(sub_face) for sub_face in cur_faces])
        cur_part_nn_verts.append(cur_nn_verts)
        cur_part_nn_face_indices.append(cur_nn_faces)
      cur_part_to_stats[cur_cat_part] = {
        'nn_verts': cur_part_nn_verts, 'nn_faces': cur_part_nn_face_indices
      }
    cat_to_stats[cur_cat] = cur_part_to_stats
  return cat_to_stats

data_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_scaled"
motion_cats = os.listdir(data_root_folder)
motion_cats = [fn for fn in motion_cats if os.path.isdir(os.path.join(data_root_folder, fn))]
cat_to_stats = get_processed_data_stats(data_root_folder, motion_cats)
np.save("/nas/datasets/gen/datasets/" + "deci_stats.npy", cat_to_stats)


In [ ]:


def get_part_nn_stats(sv_root_folder):
  motion_cats = os.listdir(sv_root_folder)
  motion_cats = [fn for fn in motion_cats if os.path.isdir(os.path.join(sv_root_folder, fn))]
  tot_part_n = 0
  for cur_cat in motion_cats:
    cur_motion_cat_folder = os.path.join(sv_root_folder, cur_cat)
    cur_cat_part_nms = os.listdir(cur_motion_cat_folder)
    cur_cat_part_nms = [fn for fn in cur_cat_part_nms if os.path.isdir(os.path.join(cur_motion_cat_folder, fn))]
    tot_part_n += len(cur_cat_part_nms)
  print(f"tot_part_n: {tot_part_n}")

data_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first"
get_part_nn_stats(data_root_folder)


In [1]:


from data_utils import save_obj_file
from data_utils import read_obj_file_ours
import numpy as np
import math


def sample_pts_from_mesh(vertices, faces, npoints=512):

    sampled_pcts = []
    pts_to_seg_idx = []
    # triangles and pts
    for i in range(len(faces)): # 
        cur_face = faces[i]
        n_tris = len(cur_face) - 2
        v_as, v_bs, v_cs = [cur_face[0] for _ in range(n_tris)], cur_face[1: len(cur_face) - 1], cur_face[2: len(cur_face)]
        for v_a, v_b, v_c in zip(v_as, v_bs, v_cs):
          
            v_a, v_b, v_c = vertices[v_a - 1], vertices[v_b - 1], vertices[v_c - 1]
            ab, ac = v_b - v_a, v_c - v_a
            cos_ab_ac = (np.sum(ab * ac) / np.clip(np.sqrt(np.sum(ab ** 2)) * np.sqrt(np.sum(ac ** 2)), a_min=1e-9,
                                                  a_max=9999999.)).item()
            sin_ab_ac = math.sqrt(min(max(0., 1. - cos_ab_ac ** 2), 1.))
            cur_area = 0.5 * sin_ab_ac * np.sqrt(np.sum(ab ** 2)).item() * np.sqrt(np.sum(ac ** 2)).item()

            cur_sampled_pts = int(cur_area * npoints)
            cur_sampled_pts = 1 if cur_sampled_pts == 0 else cur_sampled_pts
            # if cur_sampled_pts == 0:

            tmp_x, tmp_y = np.random.uniform(0, 1., (cur_sampled_pts,)).tolist(), np.random.uniform(0., 1., (
            cur_sampled_pts,)).tolist()

            for xx, yy in zip(tmp_x, tmp_y):
                sqrt_xx, sqrt_yy = math.sqrt(xx), math.sqrt(yy)
                aa = 1. - sqrt_xx
                bb = sqrt_xx * (1. - yy)
                cc = yy * sqrt_xx
                cur_pos = v_a * aa + v_b * bb + v_c * cc
                sampled_pcts.append(cur_pos)
                # pts_to_seg_idx.append(cur_tri_seg)

    # seg_idx_to_sampled_pts = {}
    sampled_pcts = np.array(sampled_pcts, dtype=np.float)

    return sampled_pcts

def normalize_object_orientation(obj_fn):
  cur_verts, cur_faces = read_obj_file_ours(obj_fn)
#   sampled_pts = sample_pts_from_mesh(cur_verts, cur_faces)
  if "voxelized" in obj_fn:
    sampled_pts = cur_verts
  else:
    sampled_pts = sample_pts_from_mesh(cur_verts, cur_faces)
  cov = np.cov(sampled_pts.T, )
  evals, evecs = np.linalg.eig(cov, )
  sort_indices = np.argsort(evals)[::-1]
  evec1 = evecs[:, sort_indices[0]] # (3,) dim vector as the largest object orientation vector
  # print(evals)
  # print(evec1) # x y z
  # print(evecs.shape, sampled_pts.shape)
  # sampled_pts_normalized = np.matmul(evecs.T, sampled_pts.T).T
  # ps.register_point_cloud(f"ori", sampled_pts, radius=0.012, color=color[0])
  # sampled_pts_normalized = sampled_pts_normalized[:, [1, 0, 2]]
  cur_verts_normalized = np.matmul(evecs.T, cur_verts.T).T
  cur_verts_normalized = cur_verts_normalized[:, [1, 0, 2]]
  return cur_verts_normalized, cur_faces


# sv_dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_50_normalized"
sv_dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_part_first_normalized"
os.makedirs(sv_dataset_root_folder, exist_ok=True)
# dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_50"
dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_part_first"

tot_motion_cats = os.listdir(dataset_root_folder)
tot_motion_cats = [fnn for fnn in tot_motion_cats if os.path.isdir(os.path.join(dataset_root_folder, fnn))]


def normalize_data(cat_folder, sv_cat_folder):
    os.makedirs(sv_cat_folder, exist_ok=True)
    part_names = os.listdir(cat_folder)
    part_names = [pn for pn in part_names if os.path.isdir(os.path.join(cat_folder, pn))]
    for pn in part_names:
        cur_part_folder = os.path.join(cat_folder, pn)
        cur_part_objs = os.listdir(cur_part_folder)
        cur_part_sv_folder = os.path.join(sv_cat_folder, pn)
        os.makedirs(cur_part_sv_folder, exist_ok=True)
        cur_part_objs = [obj_fn for obj_fn in cur_part_objs if obj_fn.endswith(".obj")]
        for obj_fn in cur_part_objs:
            cur_obj_fn = os.path.join(cur_part_folder, obj_fn)
            cur_sv_obj_fn = os.path.join(cur_part_sv_folder, obj_fn)
            try:
                cur_normalized_verts, cur_faces = normalize_object_orientation(cur_obj_fn)
                save_obj_file(cur_normalized_verts, cur_faces, cur_sv_obj_fn)
            except:
                continue    


for motion_cat in tot_motion_cats:
  print(f"Processing category {motion_cat}...")
  cur_cat_folder = os.path.join(dataset_root_folder, motion_cat)
  cur_sv_cat_folder = os.path.join(sv_dataset_root_folder, motion_cat)
  normalize_data(cur_cat_folder, cur_sv_cat_folder)

/home/xueyi/anaconda3/envs/mage/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Processing category globe...
Processing category screwdriver...
Processing category bike...
Processing category wine_bottle...
Processing category watch...
Processing category rocking_chair...
Processing category stapler...
Processing category lamp...
Processing category excavator...
Processing category car...
Processing category lighter...
Processing category tank...
Processing category seesaw...
Processing category bucket...
Processing category fan...
Processing category closestool...
Processing category washing_machine...
Processing category oven...
Processing category faucet...
Processing category cabinet...
Processing category scissors...
Processing category door...
Processing category pen...
Processing category water_bottle...
Processing category eyeglasses...
Processing category clock...


In [3]:

# import enum
import data_utils
from data_utils import save_obj_file
from data_utils import read_obj_file_ours
import numpy as np


def load_training_objs_data(training_folder, nn_max_permit_vertices=1000, nn_max_permite_faces=3000):
  tot_obj_fns = os.listdir(training_folder)
  tot_obj_fns = [fn for fn in tot_obj_fns if fn.endswith(".obj") and "context" not in fn]
  tot_obj_vertices = []
  for cur_obj in tot_obj_fns:
    cur_obj_fn = os.path.join(training_folder, cur_obj)
    cur_verts, cur_faces = read_obj_file_ours(cur_obj_fn)
    cur_verts = data_utils.center_vertices(cur_verts)

    cur_verts = data_utils.normalize_vertices_scale(cur_verts)
    # print(cur_verts.shape)
    # if cur_verts.shape[0] > nn_max_permit_vertices:
    #   continue
    tot_obj_vertices.append(cur_verts)
  return tot_obj_vertices, tot_obj_fns

def get_vertices_distances(samples_folder, training_folder):
  tot_training_obj_vertices, tot_training_obj_fns = load_training_objs_data(training_folder=training_folder)
  tot_samples_obj_vertices, tot_samples_obj_fns = load_training_objs_data(training_folder=samples_folder)
  
  train_idx_to_nns = {}
  avg_dists = []
  sample_fn_to_minn_dist_fn = {}
  tot_maxx_dist = 0.0
  tot_maxx_sample_fn = ""
  tot_maxx_training_fn = ""
  for i, cur_sample_verts in enumerate(tot_samples_obj_vertices):
    minn_avg_dist = 999999.9 
    minn_tr_idx = 0
    for i_tr, cur_train_verts in enumerate(tot_training_obj_vertices):
      dist_cur_sample_train_verts = np.sum((np.reshape(cur_sample_verts, (cur_sample_verts.shape[0], 1, 3)) - np.reshape(cur_train_verts, (1, cur_train_verts.shape[0], 3))) ** 2, axis=-1)
      dist_cur_sample_train_verts = np.sqrt(dist_cur_sample_train_verts)
      dist_sample_to_train, dist_train_to_sample = np.min(dist_cur_sample_train_verts, axis=-1), np.min(dist_cur_sample_train_verts, axis=-2)
      avg_dist = np.mean(dist_sample_to_train) + np.mean(dist_train_to_sample)
      avg_dist = avg_dist / 2.
      if minn_avg_dist > avg_dist:
        minn_avg_dist = avg_dist
        minn_tr_idx = i_tr
    sample_fn_to_minn_dist_fn[tot_samples_obj_fns[i]] = (minn_avg_dist, tot_training_obj_fns[minn_tr_idx])
    
    cur_sample_obj_tot_fn = os.path.join(samples_folder, tot_samples_obj_fns[i])
    cur_training_obj_tot_fn = os.path.join(training_folder, tot_training_obj_fns[minn_tr_idx])
    paired_data_sv_folder = "samples_paired_data"

    if minn_tr_idx not in train_idx_to_nns:
      train_idx_to_nns[minn_tr_idx] = 1
    else:
      train_idx_to_nns[minn_tr_idx] += 1
    
    samples_sv_paired_fn = os.path.join(paired_data_sv_folder, f"pair_{i}_sample.obj")
    training_sv_paired_fn = os.path.join(paired_data_sv_folder, f"pair_{i}_training.obj")
    os.system(f"cp {cur_sample_obj_tot_fn} {samples_sv_paired_fn}")
    os.system(f"cp {cur_training_obj_tot_fn} {training_sv_paired_fn}")

    if minn_avg_dist > tot_maxx_dist:
      tot_maxx_sample_fn = f"pair_{i}_sample.obj"
      tot_maxx_training_fn = f"pair_{i}_training.obj"
      tot_maxx_dist = minn_avg_dist

    avg_dists.append(minn_avg_dist)
  print(sample_fn_to_minn_dist_fn)
  print("train_idx_to_nns:", train_idx_to_nns)
  print("maxx", tot_maxx_dist, tot_maxx_sample_fn, tot_maxx_training_fn)
  return sample_fn_to_minn_dist_fn, avg_dists

os.makedirs("samples_paired_data", exist_ok=True)

# samples_folder = "/nas/datasets/gen/samples/sampling_finetune_eyeglasses_ar_part_indi_False_MotionDataset_processed_deci_part_first_30_cat_eyeglasses_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000_mask_low_0.15_high_0.16"
# samples_folder = "/nas/datasets/gen/samples/sampling_no_finetune_eyeglasses_ar_part_indi_False_MotionDataset_processed_deci_part_first_30_cat_eyeglasses_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000_mask_low_0.15_high_0.16" 
# samples_folder = "/nas/datasets/gen/samples/sampling_no_finetune_eyeglasses_ar_part_indi_False_MotionDataset_processed_deci_part_first_30_cat_eyeglasses_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000_mask_low_0.15_high_0.16"
samples_folder = "/nas/datasets/gen/samples/sampling_adapter_finetune_eyeglasses_ar_part_indi_False_MotionDataset_processed_deci_part_first_30_cat_eyeglasses_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000_mask_low_0.15_high_0.16"
samples_folder = "/nas/datasets/gen/samples/finetuning_ar_prompt_prompt_finetune_256_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_normalized_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/finetuning_ar_prompt_prompt_finetune_512_frame_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_normalized_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/finetuning_ar_prompt_prompt_finetune_512_wb_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_normalized_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/finetuning_ar_prompt_prompt_finetune_512_wb_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_normalized_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/finetuning_ar_prompt_prompt_finetune_512_oven_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_normalized_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_ctof_v2_finetune_frames_cat_part_indi_Falselight_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/finetuning_ar_prompt_prompt_finetune_512_frame_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_normalized_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_ctof_v2_some_parts_v2_to_frames_cat_part_indi_Falselight_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/finetuning_ar_ctof_v2_prompt_frames_prompt_cat_part_indi_Falselight_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/finetuning_ar_prompt_prompt_finetune_512_fr_real_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_normalized_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_prompt_prompt_finetune_eyeglasses_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_prompt_prompt_finetune_eyeglasses_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
# samples_folder = "/nas/datasets/gen/samples/uuuuuu"
samples_folder =  "/nas/datasets/gen/samples/sampling_cw_750_0_beam_False_ar_part_indi_True_MotionDataset_processed_deci_part_first_30_cat_eyeglasses_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000_mask_low_0.15_high_0.16"
samples_folder = "/nas/datasets/gen/samples/sampling_cw_750_3_beam_False_ar_part_indi_True_MotionDataset_processed_deci_part_first_30_cat_eyeglasses_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000_mask_low_0.15_high_0.16"
samples_folder = "/nas/datasets/gen/samples/finetuning_ar_prompt_prompt_ft_fr_512_ctx_750_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_normalized_reform_bsz_1_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/pretraining_ar_some_parts_v9_sd_fr_False_indi_True_ctx_750_face_cond_False_light_False_spec_False_vpred_1.0_scale_0_MotionDataset_processed_deci_part_first_30_normalized_reform_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_prompt_prompt_fr_eyeglasses_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_normalized_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
samples_folder = "/nas/datasets/gen/samples/sampling_cw_750_3_bm_beam_False_ar_part_indi_True_MotionDataset_processed_deci_part_first_30_cat_eyeglasses_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000_mask_low_0.15_high_0.16"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_prompt_prompt_fr_eye_cat_part_indi_True_face_cond_False_light_False_shift_False_filp_False_MotionDataset_processed_deci_part_first_30_normalized_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000"
# samples_folder = "/nas/datasets/gen/samples/wwww"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized/eyeglasses/none_motion"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized_reform/water_bottle/none_motion"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized_reform/eyeglasses/none_motion"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_part_first/eyeglasses/none_motion"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized_reform/screwdriver/dof_rootd_Aa001_r"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_part_first/screwdriver/dof_rootd_Aa001_r"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_part_first/eyeglasses/dof_rootd_Aa001_r"
training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_part_first/eyeglasses/none_motion"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized_reform/eyeglasses/dof_rootd_Aa001_r"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized_reform/pen/dof_rootd_Ba001_t"


_, avg_dists = get_vertices_distances(samples_folder, training_folder)

print(sum(avg_dists) / len(avg_dists))
    

{'training_step_12_part_0_ins_0.obj': (0.015599490096810744, '0003.obj'), 'training_step_36_part_0_ins_1.obj': (0.0015754354859600704, '0022.obj'), 'training_step_53_part_0_ins_0.obj': (0.003369987634153662, '0007.obj'), 'training_step_11_part_0_ins_0.obj': (0.023154302072270455, '0015.obj'), 'training_step_3_part_0_ins_1.obj': (0.0016923022435672182, '0009.obj'), 'training_step_59_part_0_ins_0.obj': (0.11205161062772559, '0028.obj'), 'training_step_1_part_0_ins_0.obj': (0.0012262901843470458, '0001.obj'), 'training_step_28_part_0_ins_1.obj': (0.015939707298299882, '0029.obj'), 'training_step_57_part_0_ins_0.obj': (0.0016923022435672182, '0009.obj'), 'training_step_37_part_0_ins_0.obj': (0.0015534536770070643, '0006.obj'), 'training_step_49_part_0_ins_1.obj': (0.007216024028834145, '0007.obj'), 'training_step_63_part_0_ins_1.obj': (0.010334327246340626, '0023.obj'), 'training_step_61_part_0_ins_0.obj': (0.03777375322488474, '0018.obj'), 'training_step_58_part_0_ins_0.obj': (0.001641857

In [2]:

# import enum
import data_utils
from data_utils import save_obj_file
from data_utils import read_obj_file_ours
import numpy as np
import binvox_rw

# def read_vertices_


def load_training_voxs_data(training_folder, nn_max_permit_vertices=1000, nn_max_permite_faces=3000, part_name="", inst_fns=None):
  tot_ins_fns = os.listdir(training_folder)
  tot_ins_fns = [fn for fn in tot_ins_fns if os.path.isdir(os.path.join(training_folder, fn))]
  # if inst_fns is not None:
    

  tot_obj_vertices = []
  tot_obj_fnnns  = []
  for cur_ins_fn in tot_ins_fns:
    cur_ins_folder = os.path.join(training_folder, cur_ins_fn)
    tot_obj_fns = os.listdir(cur_ins_folder)
    # tot_obj_fns = os.listdir(training_folder)
    
    tot_obj_fns = [fn for fn in tot_obj_fns if fn.endswith(".binvox") and 'context' not in fn]
    tot_obj_fns = [fn for fn in tot_obj_fns if part_name in fn]
    print(f"cur ins folder: {cur_ins_folder}, tot_obj_fns: {tot_obj_fns}")
    
    for cur_obj in tot_obj_fns:
      cur_vox_fn = os.path.join(cur_ins_folder, cur_obj)
      # cur_vox_fn = os.path.join(cur_shp_folder, vox_fn)
      with open(cur_vox_fn, 'rb') as f:
        model = binvox_rw.read_as_3d_array(f)
      pts = []
      dim_x, dim_y, dim_z = model.dims
      for i_x in range(dim_x):
        for i_y in range(dim_y):
            for i_z in range(dim_z):
                cur_model_data = int(model.data[i_x, i_y, i_z].item())
                if cur_model_data > 0:
                    pts.append([i_x, i_y, i_z])
      pts = np.array(pts, dtype=np.long) # n_pts x 3
      cur_verts = pts.astype(np.float32)
      cur_verts = data_utils.center_vertices(cur_verts)

      cur_verts = data_utils.normalize_vertices_scale(cur_verts)
      # print(cur_verts.shape)
      # if cur_verts.shape[0] > nn_max_permit_vertices:
      #   continue
      tot_obj_vertices.append(cur_verts)
      tot_obj_fnnns.append(cur_obj)
  print(f"tot_obj_vertices: {len(tot_obj_vertices)}")
  # return tot_obj_vertices, tot_obj_fns
  return tot_obj_vertices, tot_obj_fnnns


def load_training_objs_data(training_folder, nn_max_permit_vertices=1000, nn_max_permite_faces=3000):
  tot_obj_fns = os.listdir(training_folder)
  tot_obj_fns = [fn for fn in tot_obj_fns if fn.endswith(".obj") and "context" not in fn]
  tot_obj_vertices = []
  for cur_obj in tot_obj_fns:
    cur_obj_fn = os.path.join(training_folder, cur_obj)
    cur_verts, cur_faces = read_obj_file_ours(cur_obj_fn)
    cur_verts = data_utils.center_vertices(cur_verts)

    cur_verts = data_utils.normalize_vertices_scale(cur_verts)
    # cur_verts = cur_verts[:, [2, 1, 0]]
    # print(cur_verts.shape)
    # if cur_verts.shape[0] > nn_max_permit_vertices:
    #   continue
    tot_obj_vertices.append(cur_verts)
  return tot_obj_vertices, tot_obj_fns



def get_voxs_distances(samples_folder, training_folder, part_name=""):
  tot_training_obj_vertices, tot_training_obj_fns = load_training_voxs_data(training_folder=training_folder, part_name=part_name)
  tot_samples_obj_vertices, tot_samples_obj_fns = load_training_objs_data(training_folder=samples_folder)
  
  train_idx_to_nns = {}
  avg_dists = []
  sample_fn_to_minn_dist_fn = {}
  tot_maxx_dist = 0.0
  tot_maxx_sample_fn = ""
  tot_maxx_training_fn = ""
  for i, cur_sample_verts in enumerate(tot_samples_obj_vertices):
    minn_avg_dist = 999999.9 
    minn_tr_idx = 0
    for i_tr, cur_train_verts in enumerate(tot_training_obj_vertices):
      dist_cur_sample_train_verts = np.sum((np.reshape(cur_sample_verts, (cur_sample_verts.shape[0], 1, 3)) - np.reshape(cur_train_verts, (1, cur_train_verts.shape[0], 3))) ** 2, axis=-1)
      dist_cur_sample_train_verts = np.sqrt(dist_cur_sample_train_verts)
      dist_sample_to_train, dist_train_to_sample = np.min(dist_cur_sample_train_verts, axis=-1), np.min(dist_cur_sample_train_verts, axis=-2)
      avg_dist = np.mean(dist_sample_to_train) + np.mean(dist_train_to_sample)
      avg_dist = avg_dist / 2.
      if minn_avg_dist > avg_dist:
        minn_avg_dist = avg_dist
        minn_tr_idx = i_tr
    sample_fn_to_minn_dist_fn[tot_samples_obj_fns[i]] = (minn_avg_dist, tot_training_obj_fns[minn_tr_idx])
    
    cur_sample_obj_tot_fn = os.path.join(samples_folder, tot_samples_obj_fns[i])
    cur_training_obj_tot_fn = os.path.join(training_folder, tot_training_obj_fns[minn_tr_idx])
    paired_data_sv_folder = "samples_paired_data"

    if minn_tr_idx not in train_idx_to_nns:
      train_idx_to_nns[minn_tr_idx] = 1
    else:
      train_idx_to_nns[minn_tr_idx] += 1
    
    samples_sv_paired_fn = os.path.join(paired_data_sv_folder, f"pair_{i}_sample.obj")
    training_sv_paired_fn = os.path.join(paired_data_sv_folder, f"pair_{i}_training.obj")
    os.system(f"cp {cur_sample_obj_tot_fn} {samples_sv_paired_fn}")
    # os.system(f"cp {cur_training_obj_tot_fn} {training_sv_paired_fn}")

    cur_training_verts = tot_training_obj_vertices[minn_tr_idx]
    save_obj_file(cur_training_verts, [], training_sv_paired_fn)
# 
    if minn_avg_dist > tot_maxx_dist:
      tot_maxx_sample_fn = f"pair_{i}_sample.obj"
      tot_maxx_training_fn = f"pair_{i}_training.obj"
      tot_maxx_dist = minn_avg_dist

    avg_dists.append(minn_avg_dist)
  print(sample_fn_to_minn_dist_fn)
  print("train_idx_to_nns:", train_idx_to_nns)
  print("maxx", tot_maxx_dist, tot_maxx_sample_fn, tot_maxx_training_fn)
  return sample_fn_to_minn_dist_fn, avg_dists

os.makedirs("samples_paired_data", exist_ok=True)

##### water bottle samples #####
samples_folder = "/nas/datasets/gen/samples/pretraining_ar_grid_v1_vox_3_cv_2_fr_False_indi_True_ctx_-1_face_cond_False_light_False_spec_False_vpred_1.0_scale_0_MotionDataset_processed_voxelized_64_bsz_1_max_verts_200000_faces_3000_max_permit_verts_200000_faces_3000"
# samples_folder = "/nas/datasets/gen/samples/pretraining_ar_grid_v1_vox_3_fr_False_indi_False_ctx_-1_face_cond_False_light_False_spec_False_vpred_1.0_scale_0_MotionDataset_processed_voxelized_64_bsz_1_max_verts_200000_faces_3000_max_permit_verts_200000_faces_3000"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_grid_v1_vox_3_cv_4_fr_False_indi_True_ctx_-1_gs_900_vpred_1.0_scale_0_MotionDataset_processed_voxelized_64_bsz_1_max_verts_900_max_permit_verts_900"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_grid_eye_scale_fr_False_indi_True_ctx_-1_gs_900_vpred_1.0_scale_0_MotionDataset_processed_voxelized_64_bsz_1_max_verts_900_max_permit_verts_900"

samples_folder = "/nas/datasets/gen/samples/sampling_ar_vox_prompt_v9_parts_indi_True_ctx_300_gs_1000_class_62_scale_0_MotionDataset_processed_voxelized_64_bsz_1_max_verts_1000_max_permit_verts_1000_key_len_32"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_vox_prompt_parts_v9_frames_pr_v3_indi_True_ctx_400_gs_800_scale_0_MotionDataset_processed_voxelized_64_v9_ns_50_bsz_1_max_verts_800_max_permit_verts_800_key_len_512"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_grid_vocab_grid_vocab_legs_n3_indi_True_ctx_-1_gs_2_ngrids_700_scale_0_MotionDataset_processed_voxelized_64_v9_ns_50_bsz_1_max_verts_6400_max_permit_verts_6400_key_256"
samples_folder = "/nas/datasets/gen/samples/pretraining_ar_vox_prompt_parts_v9_6n_indi_False_ctx_300_gs_800_scale_0_MotionDataset_processed_voxelized_64_ns_50_bsz_1_max_verts_800_max_permit_verts_800_key_len_512"
samples_folder = "/nas/datasets/gen/samples/pretraining_ar_vox_prompt_parts_v9_6n_indi_False_ctx_300_gs_650_scale_0_MotionDataset_processed_voxelized_64_ns_50_bsz_1_max_verts_650_max_permit_verts_650_key_len_512"
samples_folder = "/nas/datasets/gen/samples/sampling_ar_vox_prompt_parts_v9_uncond_indi_False_ctx_300_gs_650_class_0_scale_0_MotionDataset_processed_voxelized_64_ns_50_bsz_1_max_verts_650_max_permit_verts_650_key_len_512"

training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64/water_bottle"
training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64/pen"
# dof_rootd_Aa001_r
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64/handcart"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64/eyeglasses"  # MotionDataset_processed_voxelized_64_v9_ns_50
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64_v9_ns_50/scissors"  # MotionDataset_processed_voxelized_64_v9_ns_50
training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64_v9_ns_50/eyeglasses"
training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64_v9_ns_50/eyeglasses"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64_ns_50/scissors"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64/pen"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized_reform/eyeglasses/dof_rootd_Aa001_r"
# training_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized_reform/pen/dof_rootd_Ba001_t"


# _, avg_dists = get_voxs_distances(samples_folder, training_folder, part_name="none_motion")
_, avg_dists = get_voxs_distances(samples_folder, training_folder, part_name="dof_rootd_Aa001_r")

print(sum(avg_dists) / len(avg_dists))
    

cur ins folder: /nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64_v9_ns_50/eyeglasses/0030, tot_obj_fns: ['dof_rootd_Aa001_r_s_47.obj_64.binvox', 'dof_rootd_Aa001_r_s_32.obj_64.binvox', 'dof_rootd_Aa001_r_s_43.obj_64.binvox', 'dof_rootd_Aa001_r_s_37.obj_64.binvox', 'dof_rootd_Aa001_r_s_34.obj_64.binvox', 'dof_rootd_Aa001_r_s_25.obj_64.binvox', 'dof_rootd_Aa001_r_s_17.obj_64.binvox', 'dof_rootd_Aa001_r_s_28.obj_64.binvox', 'dof_rootd_Aa001_r_s_10.obj_64.binvox', 'dof_rootd_Aa001_r_s_13.obj_64.binvox', 'dof_rootd_Aa001_r_s_23.obj_64.binvox', 'dof_rootd_Aa001_r_s_46.obj_64.binvox', 'dof_rootd_Aa001_r_s_27.obj_64.binvox', 'dof_rootd_Aa001_r_s_50.obj_64.binvox', 'dof_rootd_Aa001_r_s_12.obj_64.binvox', 'dof_rootd_Aa001_r_s_20.obj_64.binvox', 'dof_rootd_Aa001_r_s_9.obj_64.binvox', 'dof_rootd_Aa001_r_s_29.obj_64.binvox', 'dof_rootd_Aa001_r_s_18.obj_64.binvox', 'dof_rootd_Aa001_r_s_6.obj_64.binvox', 'dof_rootd_Aa001_r_s_15.obj_64.binvox', 'dof_rootd_Aa001_r_s_11.obj_64.binvox', 'd

In [1]:


import numpy as np
def get_part_indicator_info(fn):
  indi_to_cat_idx = np.load(fn, allow_pickle=True).item()
  print(indi_to_cat_idx)
  
fn = "/nas/datasets/gen/ckpts/pretraining_ar_some_parts_v0_cond_cut_verts_fr_False_indi_True_ctx_750_face_cond_False_light_False_spec_False_vpred_1.0_scale_0_MotionDataset_processed_deci_part_first_30_normalized_reform_bsz_2_max_verts_400_faces_2000_max_permit_verts_400_faces_2000/dataset_train_part_indicator_to_idx.npy"
fn = "/nas/datasets/gen/ckpts/pretraining_ar_grid_v9_parts_fr_False_indi_True_ctx_400_gs_900_vpred_1.0_scale_1_MotionDataset_processed_voxelized_64_bsz_1_max_verts_900_max_permit_verts_900/dataset_train_part_indicator_to_idx.npy"
fn = "/nas/datasets/gen/ckpts/pretraining_ar_vox_prompt_parts_v9_indi_True_ctx_400_gs_900_scale_0_MotionDataset_processed_voxelized_64_bsz_1_max_verts_900_max_permit_verts_900_key_len_32/dataset_train_part_indicator_to_idx.npy"
fn = "/nas/datasets/gen/ckpts/pretraining_ar_vox_prompt_parts_v9_indi_True_ctx_400_gs_800_scale_0_MotionDataset_processed_voxelized_64_v9_ns_50_bsz_1_max_verts_800_max_permit_verts_800_key_len_512/dataset_train_part_indicator_to_idx.npy"
fn = "/nas/datasets/gen/ckpts/pretraining_ar_grid_vocab_gv_n5_motion_p3_indi_False_ctx_600_gs_2_ngrids_800_nobj_3_scale_0_MotionDataset_processed_voxelized_64_ns_50_bsz_1_max_verts_6500_max_permit_verts_6500_key_256/dataset_train_part_indicator_to_idx.npy"
get_part_indicator_info(fn)
# 0, 3, 10, 25, 37, 46, 44, 51, 57
# 3, 56, 62
# 0 1 3 7
# 0 8 5 1
# /nas/datasets/gen/ckpts/pretraining_ar_grid_vocab_grid_vox_vocab_n3_mp_indi_True_ctx_-1_gs_2_ngrids_800_scale_0_MotionDataset_processed_voxelized_64_v9_ns_50_bsz_1_max_verts_400_max_permit_verts_400_key_256/vertex_model/20000.pth
# .join(vertex_model_path.split('/')[:-2])

{'globe-none_motion': 0, 'globe-dof_rootd_Ca001_r': 1, 'globe-dof_rootd_Ba001_r': 2, 'valve-dof_rootd_Aa001_r': 3, 'valve-none_motion': 4, 'handcart-dof_rootd_Aa002_r': 5, 'handcart-dof_rootd_Aa003_r': 6, 'handcart-dof_rootd_Aa001_r': 7, 'handcart-dof_rootd_Aa004_r': 8, 'handcart-none_motion': 9, 'screwdriver-dof_rootd_Aa001_r': 10, 'window-dof_rootd_Aa002_r': 11, 'window-dof_rootd_Aa001_r': 12, 'window-none_motion': 13, 'window-dof_rootd_Aa001_t': 14, 'window-dof_rootd_Aa002_t': 15, 'window-dof_rootd_Aa003_r': 16, 'window-dof_rootd_Aa004_t': 17, 'window-dof_rootd_Aa005_t': 18, 'refrigerator-dof_rootd_Aa002_r': 19, 'refrigerator-dof_rootd_Aa003_r': 20, 'refrigerator-none_motion': 21, 'refrigerator-dof_rootd_Aa001_r': 22, 'refrigerator-dof_rootd_Ba002_t': 23, 'refrigerator-dof_rootd_Ba001_t': 24, 'refrigerator-dof_rootd_Aa003_t': 25, 'refrigerator-dof_rootd_Aa001_t': 26, 'refrigerator-dof_rootd_Aa002_t': 27, 'refrigerator-dof_rootd_Ba003_t': 28, 'swiss_army_knife-dof_rootd_Ba001_r': 29,

In [21]:

import math

train_idx_to_nn = {3: 8, 21: 7, 16: 8, 6: 5, 41: 7, 28: 10, 23: 8, 35: 26, 15: 8, 33: 4, 22: 7, 30: 6, 4: 5, 14: 9, 26: 5, 27: 4, 9: 5, 24: 4}
train_idx_to_nn = {22: 7, 33: 4, 21: 8, 6: 6, 35: 9, 16: 8, 15: 11, 3: 3, 14: 4, 27: 10, 30: 6, 4: 4, 41: 6, 26: 11, 28: 5, 9: 3, 23: 3, 24: 6, 29: 1, 36: 1}
train_idx_to_nn = {22: 7, 15: 13, 14: 4, 35: 15, 30: 12, 23: 7, 21: 9, 33: 15, 27: 7, 3: 7, 17: 2, 4: 4, 41: 10, 28: 13, 24: 5, 16: 13, 26: 8, 9: 5, 18: 1, 6: 3}
train_idx_to_nn = {362: 1, 370: 3, 377: 1, 395: 1, 365: 1, 771: 1, 404: 1, 380: 1, 2080: 1, 379: 1, 399: 1, 400: 1, 360: 1, 374: 2, 1963: 1, 615: 1, 349: 1}
sorted_items = sorted(train_idx_to_nn.items(), key=lambda ii: ii[1], reverse=True)
print(len(sorted_items))
print(sorted_items)
tot_keys = [ii[0] for ii in sorted_items]
tot_vals = [ii[1] for ii in sorted_items]
tot_n_vals = sum(tot_vals)
tot_norm_vals = [float(ii) / float(tot_n_vals) for ii in tot_vals]
print(tot_norm_vals)
entropy = [-math.log(ii) * ii for ii in tot_norm_vals]
entropy = sum(entropy)
print("entropy", entropy)

17
[(370, 3), (374, 2), (362, 1), (377, 1), (395, 1), (365, 1), (771, 1), (404, 1), (380, 1), (2080, 1), (379, 1), (399, 1), (400, 1), (360, 1), (1963, 1), (615, 1), (349, 1)]
[0.15, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
entropy 2.76162571219778


In [3]:
# category_name: {part_name: {instance_name_with_scale: points_num}}}


import data_utils
import torch
import numpy as np

def get_sampled_points(obj_fn, nn_y_intervals=10, nn_per_verts_samples=10):
  # cur_verts: n_verts x 3; cur_faces: faces...
  cur_verts, cur_faces = data_utils.read_obj_file_ours(obj_fn=obj_fn)
  # 
  if len(cur_verts.shape) == 1:
    cur_verts = np.reshape(cur_verts, (1, 3))
  cur_verts_max, cur_verts_min = np.max(cur_verts, axis=0), np.min(cur_verts, axis=0) # cur verts for 
  cur_verts_max_y, cur_verts_min_y = cur_verts_max[1].item(), cur_verts_min[1].item() # max and min of verts
  len_y_interval = (cur_verts_max_y - cur_verts_min_y) / float(nn_y_intervals)
  sampled_pts = []
  for i_inter in range(nn_y_intervals - 1):
    cur_sampled_y = cur_verts_min_y + (i_inter + 1) * len_y_interval # sampled_y
    cur_y_sample_pts = []
    for cur_face in cur_faces:
      cur_face_verts = [cur_verts[v_idx - 1] for v_idx in cur_face] # vertices coordinates of the face
      # cur_face_verts
      v_a, v_b, v_c = cur_face_verts[:3]
      v_ab, v_ac = v_b - v_a, v_c - v_a
      v_n = np.cross(v_ab, v_ac) # (3,)
      a, b, c = v_n[0].item(), v_n[1].item(), v_n[2].item()
      # a * x + b * y + c * z = 0
      # a * x + c * z = -b * y
      # b_mult_y
      b_mult_y = -b * cur_sampled_y # .item() # cur_sampled_y.item()
      # a, b, c
      # 
      # a1 * x + b1 * y + c1 * z = 0
      # Q learning
      # x = (-b * y - c * z) / a
      # b_mult_y
      # (x1, y1, z1), (x2, y2, z2)
      intersected_verts = []
      for i_st in range(len(cur_face_verts)):
        i_ed = (i_st + 1) % len(cur_face_verts)
        st_vert, ed_vert = cur_face_verts[i_st], cur_face_verts[i_ed]
        max_vert_y, min_vert_y = st_vert[1].item(), ed_vert[1].item()
        max_vert_y, min_vert_y = max(max_vert_y, min_vert_y), min(max_vert_y, min_vert_y)

        max_vert_x, min_vert_x = st_vert[0].item(), ed_vert[0].item()
        max_vert_x, min_vert_x = max(max_vert_x, min_vert_x), min(max_vert_x, min_vert_x)

        max_vert_z, min_vert_z = st_vert[2].item(), ed_vert[2].item()
        max_vert_z, min_vert_z = max(max_vert_z, min_vert_z), min(max_vert_z, min_vert_z)
        if not (cur_sampled_y >= min_vert_y and cur_sampled_y <= max_vert_y):
          continue
        t_val = (cur_sampled_y - min_vert_y) / max(max_vert_y - min_vert_y, 1e-9)
        cur_x, cur_z = min_vert_x + t_val * (max_vert_x - min_vert_x), min_vert_z + t_val * (max_vert_z - min_vert_z)
        # (cur_x, cur_sampled_y, cur_z)
        intersected_verts.append([cur_x, cur_sampled_y, cur_z])
      intersected_verts = np.array(intersected_verts, dtype=np.float32) # .float()
      # print(f"length of intersected vertices: {len(intersected_verts)}")
      if intersected_verts.shape[0] == 0:
        continue
      a_vert, b_vert = intersected_verts[0], intersected_verts[1]
      ### 
      ab_dis = b_vert - a_vert
      # ab_len = np.(ab_dis, p=2)
      ab_len = np.sqrt(np.sum(ab_dis ** 2))
      # ab_dis = ab_dis / np.norm(ab_dis, p=2) # tot_unit_vector ---
      ab_dis = ab_dis / np.sqrt(np.sum(ab_dis ** 2))
      cur_interval_sample_pts = []
      for i_s in range(nn_per_verts_samples):
        cur_interval_cur_sample_pt = a_vert + ab_dis * (i_s) * ab_len / nn_per_verts_samples
        cur_interval_sample_pts.append(cur_interval_cur_sample_pt)
      cur_interval_sample_pts = np.array(cur_interval_sample_pts, dtype=np.float32) # n_tot_samples x 3
      cur_y_sample_pts.append(cur_interval_sample_pts)
    if len(cur_y_sample_pts) > 0:
      cur_y_sample_pts = np.concatenate(cur_y_sample_pts, axis=0) # tot_n_y_sample x 3
      sampled_pts.append(cur_y_sample_pts)
  
  if len(sampled_pts) > 0:
    sampled_pts = np.concatenate(sampled_pts, axis=0)
  else:
    sampled_pts = np.zeros((0, 3), dtype=np.float32)

  
  tot_res_verts_pts = np.concatenate([cur_verts, sampled_pts], axis=0)

  tot_res_verts_pts_torch = torch.from_numpy(tot_res_verts_pts).float()
  fps_idx = data_utils.farthest_point_sampling(tot_res_verts_pts_torch.unsqueeze(0), n_sampling=min(400, tot_res_verts_pts_torch.size(0)))
  tot_res_verts_pts_torch = tot_res_verts_pts_torch[fps_idx] # sampled vertices
  
  tot_res_verts_pts = tot_res_verts_pts_torch.numpy()
  dist_cur_verts_tot_res_verts = np.sum((np.reshape(cur_verts, (cur_verts.shape[0], 1, 3)) - np.reshape(tot_res_verts_pts, (1, tot_res_verts_pts.shape[0], 3)) ) ** 2, axis=-1) # cur_verts.shape[0] x tot_res_verts_pts.shape[0]
  minn_cur_verts_tot_res_verts_idx = np.argmin(dist_cur_verts_tot_res_verts, axis=-1).tolist() # cur_verts.ahepe[0]
  # cur_vert_idx_to_tot_vert_idx: 
  cur_vert_idx_to_tot_vert_idx = {ii + 1: kk + 1 for ii, kk in enumerate(minn_cur_verts_tot_res_verts_idx)}
  tot_res_faces = []
  for cur_face in cur_faces:
    cur_res_face = [cur_vert_idx_to_tot_vert_idx[f_idx] for f_idx in cur_face]
    tot_res_faces.append(cur_res_face)
  cur_faces = tot_res_faces
  return tot_res_verts_pts, cur_faces

def reform_meshes(fr_data_folder, sv_data_folder):
  tot_part_nms = os.listdir(fr_data_folder)
  tot_part_nms = [fn for fn in tot_part_nms if os.path.isdir(os.path.join(fr_data_folder, fn))]
  for cur_part_nm in tot_part_nms:
    print(f" Processing part name: {cur_part_nm}...")
    cur_part_folder = os.path.join(fr_data_folder, cur_part_nm)
    cur_part_folder_sv = os.path.join(sv_data_folder, cur_part_nm)
    os.makedirs(cur_part_folder_sv, exist_ok=True)
    tot_obj_fns = os.listdir(cur_part_folder)
    tot_obj_fns = [fn for fn in tot_obj_fns if fn.endswith(".obj")]
    for obj_fn in tot_obj_fns:
      cur_fr_obj_fn = os.path.join(cur_part_folder, obj_fn)
      cur_sv_obj_fn = os.path.join(cur_part_folder_sv, obj_fn)
      res_verts_pts, faces = get_sampled_points(cur_fr_obj_fn, nn_y_intervals=10, nn_per_verts_samples=10)
      # sv_obj_fn = "tmp.obj"
      data_utils.save_obj_file(res_verts_pts, faces, cur_sv_obj_fn)
  

# # get_sampled_points(obj_fn, nn_y_intervals=10, nn_per_verts_samples=10):
# obj_fn = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized/screwdriver/dof_rootd_Aa001_r/0006.obj"
# res_verts_pts, faces = get_sampled_points(obj_fn, nn_y_intervals=10, nn_per_verts_samples=10)
# sv_obj_fn = "tmp.obj"
# data_utils.save_obj_file(res_verts_pts, faces, sv_obj_fn)

fr_rt_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized"
sv_rt_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized_reform"
os.makedirs(sv_rt_folder, exist_ok=True)
tot_motion_cats = os.listdir(fr_rt_folder)
tot_motion_cats = [fn for fn in tot_motion_cats if os.path.isdir(os.path.join(fr_rt_folder, fn)) and fn != "globe" and fn != "screwdriver"]
sv_tot_motion_cats = os.listdir(sv_rt_folder)
sv_tot_motion_cats = [fn for fn in sv_tot_motion_cats if os.path.isdir(os.path.join(fr_rt_folder, fn))]
tot_motion_cats = [fn for fn in tot_motion_cats if fn not in sv_tot_motion_cats]
for cur_motion_cat in tot_motion_cats:
  print(f"Processing motion category: {cur_motion_cat}")
  cur_motion_cat_sv = os.path.join(sv_rt_folder, cur_motion_cat)
  os.makedirs(cur_motion_cat_sv, exist_ok=True)
  cur_motion_cat_fr = os.path.join(fr_rt_folder, cur_motion_cat)
  reform_meshes(cur_motion_cat_fr, cur_motion_cat_sv)



Processing motion category: rocking_chair
 Processing part name: dof_rootd_Aa001_r...
 Processing part name: none_motion...
Processing motion category: stapler
 Processing part name: dof_rootd_Ba001_r...
 Processing part name: dof_rootd_Aa001_r...
 Processing part name: none_motion...
Processing motion category: lamp
 Processing part name: dof_rootd_Ab001_r...
 Processing part name: dof_Ab001_Aa001_r...


/home/xueyi/anaconda3/envs/mage/lib/python3.6/site-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in true_divide


 Processing part name: dof_rootd_Ac001_r...
 Processing part name: dof_rootd_Ad001_r...
 Processing part name: dof_rootd_Ae001_r...
 Processing part name: dof_rootd_Aa001_r...
 Processing part name: dof_Ad001_Ac001_r...
 Processing part name: none_motion...
 Processing part name: dof_Ac001_Ab001_r...
 Processing part name: dof_Ae001_Ad001_r...
Processing motion category: excavator
 Processing part name: dof_rootd_Ab001_r...
 Processing part name: dof_Ab001_Aa001_r...
 Processing part name: dof_rootd_Ac001_r...
 Processing part name: dof_rootd_Ba001_r...
 Processing part name: dof_rootd_Ad001_r...
 Processing part name: dof_rootd_Ae001_r...
 Processing part name: dof_Ad001_Ac001_r...
 Processing part name: dof_rootd_Ba004_r...
 Processing part name: none_motion...
 Processing part name: dof_rootd_Ba002_r...
 Processing part name: dof_Ac001_Ab001_r...
 Processing part name: dof_rootd_Ba003_r...
 Processing part name: dof_Ae001_Ad001_r...
Processing motion category: car
 Processing part n

In [3]:

from quad_mesh_simplify import simplify_mesh
import data_utils
import numpy as np

def test_mesh_simplification(obj_fn):
  cur_vertices, cur_faces = data_utils.read_obj_file_ours(obj_fn=obj_fn)
  cur_faces = np.array(cur_faces, dtype=np.uint32)
  print(f"cur_facses; {cur_faces.shape}")
  new_positions, new_face = simplify_mesh(cur_vertices, cur_faces, cur_vertices.shape[0] // 5, threshold=0.5)
  new_faces = [new_face[i] for i in range(new_face.shape[0])]
  tmp_sv_fn = "tmp.obj"
  data_utils.save_obj_file(new_positions, new_faces, tmp_sv_fn)

obj_fn = "/nas/datasets/gen/datasets/MotionDataset_processed/eyeglasses/0006/none_motion.obj"
test_mesh_simplification(obj_fn=obj_fn)

: 

: 

In [12]:

import data_utils

import numpy as np
def reprocess_mesh(verts, faces):
  vert_to_new_idx = {}
  cur_vert_idx = 1
  new_verts = []
  for cur_face in faces:
    for v_idx in cur_face:
      if v_idx not in vert_to_new_idx:
        vert_to_new_idx[v_idx] = cur_vert_idx
        cur_vert_idx += 1
        new_verts.append(verts[v_idx - 1])
  new_faces = []
  for cur_face in faces:
    cur_new_face = [vert_to_new_idx[v_idx] for v_idx in cur_face]
    new_faces.append(cur_new_face)
  new_verts = np.array(new_verts, dtype=np.float32)
  return new_verts, new_faces

def preprocess_shapenet_data_process(cat_folder, sv_cat_folder):
  ins_folders = os.listdir(cat_folder)
  # 
  os.system(f"rm -r {cat_folder}/*.mtl")
  ins_folders = [fn for fn in ins_folders if fn.endswith(".obj")]
  for ins in ins_folders:
    cur_ins_sv_folder = os.path.join(sv_cat_folder, ins)
    # os.makedirs(cur_ins_sv_folder, exist_ok=True)
    cur_ins_folder = os.path.join(cat_folder, ins)
    
    # cur_ins_part_obj_fns = os.listdir(cur_ins_folder)

    # for cur_part_obj_fn in cur_ins_part_obj_fns:
    #   if not cur_part_obj_fn.endswith(".obj"):
    #     continue
    tot_cur_part_obj_fn = cur_ins_folder
    cur_ins_verts, cur_ins_faces = data_utils.read_obj_file_ours(tot_cur_part_obj_fn)
    
    new_ins_verts, new_ins_faces = reprocess_mesh(cur_ins_verts, cur_ins_faces)
    tot_cur_part_obj_sv_fn = cur_ins_sv_folder
    data_utils.save_obj_file(new_ins_verts, new_ins_faces, tot_cur_part_obj_sv_fn)


def preprocess_shapenet_data(cat_folder, sv_cat_folder):
  ins_folders = os.listdir(cat_folder)
  # 
  os.system(f"rm -r {cat_folder}/*.mtl")
  ins_folders = [fn for fn in ins_folders if fn.endswith(".obj")]
  for ins in ins_folders:
    cur_ins_sv_folder = os.path.join(sv_cat_folder, ins)
    # os.makedirs(cur_ins_sv_folder, exist_ok=True)
    cur_ins_folder = os.path.join(cat_folder, ins)
    
    # cur_ins_part_obj_fns = os.listdir(cur_ins_folder)

    # for cur_part_obj_fn in cur_ins_part_obj_fns:
    #   if not cur_part_obj_fn.endswith(".obj"):
    #     continue
    tot_cur_part_obj_fn = cur_ins_folder
    cur_ins_verts, cur_ins_faces = data_utils.read_obj_file_ours(tot_cur_part_obj_fn)
    
    tot_cur_part_obj_sv_fn = cur_ins_sv_folder
    data_utils.save_obj_file(cur_ins_verts, cur_ins_faces, tot_cur_part_obj_sv_fn)


# dataset_root = "/nas/datasets/gen/datasets"
# motion_category = "eyeglasses"
# cat_folder = "MotionDataset"
# cat_folder = os.path.join(dataset_root, cat_folder)
# sv_cat_folder = "MotionDataset_processed"
# sv_cat_folder = os.path.join(dataset_root, sv_cat_folder)
# os.makedirs(sv_cat_folder, exist_ok=True)

# tot_motion_cats_processed = os.listdir(sv_cat_folder)
# tot_motion_cats_processed = [fnn for fnn in tot_motion_cats_processed if os.path.isdir(os.path.join(sv_cat_folder, fnn))]


# tot_motion_cats = os.listdir(cat_folder)
# # tot_motion_cats = [fnn for fnn in tot_motion_cats if os.path.isdir(os.path.join(cat_folder, fnn)) and fnn != "eyeglasses"]
# tot_motion_cats = [fnn for fnn in tot_motion_cats if os.path.isdir(os.path.join(cat_folder, fnn))]

# tot_motion_cats = [fnn for fnn in tot_motion_cats if fnn not in tot_motion_cats_processed]

# tot_motion_cats = ["lamp", "screwdriver", "seesaw", "pen"]


motion_cat = "screwdriver"
part_name = "dof_rootd_Aa001_r"

# subd_root_folder = os.path.join("/home/xueyi", "gen", "neuralSubdiv", "data_meshes", "{}-{}-valid".format(motion_cat, part_name))
subd_root_folder = os.path.join("/home/xueyi", "gen", "neuralSubdiv", "data_meshes", "bob_1")

subd_folders = os.listdir(subd_root_folder)
subd_folders = [fn for fn in subd_folders if os.path.isdir(os.path.join(subd_root_folder, fn))]

for subd_folder in subd_folders:
  cur_sub_folder = os.path.join(subd_root_folder, subd_folder)
  # preprocess_shapenet_data(cur_sub_folder, cur_sub_folder)
  preprocess_shapenet_data_process(cur_sub_folder, cur_sub_folder)

# for motion_category in tot_motion_cats:
#   print(f"Preprocessing category {motion_category}...")
#   cur_cat_folder = os.path.join(cat_folder, motion_category)
#   cur_sv_cat_folder = os.path.join(sv_cat_folder, motion_category)
#   os.makedirs(cur_sv_cat_folder, exist_ok=True)
#   preprocess_shapenet_data(cur_cat_folder, cur_sv_cat_folder)

In [8]:


def modify_file_names(folder):
  obj_fns = os.listdir(folder)
  obj_fns = [fn for fn in obj_fns if fn.endswith(".obj")]
  for obj_fn in obj_fns:
    cur_obj_fn = os.path.join(folder, obj_fn)
    modified_fn = "{:03d}".format(int(obj_fn.split(".")[0])) #### modified file names...
    print(modified_fn)
    cur_modified_fn = os.path.join(folder, modified_fn + ".obj")
    os.system(f"mv {cur_obj_fn} {cur_modified_fn}")

# tot_data_nms = ["cactus_1", "fish_1", "horse_1", "spot_1"]
tot_data_nms = ["screw_2"]
# tot_data_nms = [nm + ""]
data_root_folder = "/nas/datasets/gen/datasets/subd_mesh/"
for data_nm in tot_data_nms:
  # cur_data_root_folder = os.path.join(data_root_folder, data_nm)
  cur_data_neural_subd_folder = os.path.join(data_root_folder, f"{data_nm}_neural_subdiv")
  # os.makedirs
  sub_folders = os.listdir(cur_data_neural_subd_folder)
  sub_folders = [fn for fn in sub_folders if os.path.isdir(os.path.join(cur_data_neural_subd_folder, fn))]
  for sub_folder in sub_folders:
    cur_tot_sub_folder = os.path.join(cur_data_neural_subd_folder, sub_folder)
    modify_file_names(cur_tot_sub_folder)
  # modify_file_names(cur_data_neural_subd_folder)


  
  
# root_folder = "/home/xueyi/gen/neuralSubdiv/data_meshes/screw_1"
# # root_folder = "/home/xueyi/gen/neuralSubdiv/data_meshes/screw_1"
# root_folder = "/nas/datasets/gen/datasets/subd_mesh/bob_1_neural_subdiv"
# sub_folders = os.listdir(root_folder)
# sub_folders = [fn for fn in sub_folders if os.path.isdir(os.path.join(root_folder, fn))]
# for fn in sub_folders:
#   cur_sub_folder = os.path.join(root_folder, fn)
#   modify_file_names(cur_sub_folder)

FileNotFoundError: [Errno 2] No such file or directory: '/nas/datasets/gen/datasets/subd_mesh/screw_2_neural_subdiv'

In [34]:



import data_utils
import torch
import numpy as np

def get_sampled_points(obj_fn, nn_y_intervals=10, nn_per_verts_samples=10): # per verts samples # per verts sampls; # per verts sample
  # cur_verts: n_verts x 3; cur_faces: faces...
  quantization_bits = 8
  resolution = 10
  cur_verts, cur_faces = data_utils.read_obj_file_ours(obj_fn=obj_fn)
  # 
  if len(cur_verts.shape) == 1:
    cur_verts = np.reshape(cur_verts, (1, 3))
  cur_verts_max, cur_verts_min = np.max(cur_verts, axis=0), np.min(cur_verts, axis=0) # cur verts for 
  cur_verts_max_y, cur_verts_min_y = cur_verts_max[1].item(), cur_verts_min[1].item() # max and min of verts
  len_y_interval = (cur_verts_max_y - cur_verts_min_y) / float(nn_y_intervals)

  cur_verts = cur_verts - np.mean(cur_verts, axis=0, keepdims=True)
  cur_verts = data_utils.normalize_vertices_scale(cur_verts)
  cur_verts = data_utils.quantize_verts(cur_verts) # n x 3
  
  cur_verts_max, cur_verts_min = np.max(cur_verts, axis=0), np.min(cur_verts, axis=0)
  cur_verts_max_y, cur_verts_min_y = cur_verts_max[1].item(), cur_verts_min[1].item() # verts_max and verts_min
  # 10, 20, 30
  sampled_pts = []
  for cur_sampled_y in range(0, 2 ** quantization_bits, resolution):
    if cur_sampled_y < cur_verts_min_y or cur_sampled_y > cur_verts_max_y:
      continue
    cur_sampled_y = float(cur_sampled_y)
  # sampled_pts = []
  # for i_inter in range(nn_y_intervals - 1):
  #   cur_sampled_y = cur_verts_min_y + (i_inter + 1) * len_y_interval # sampled_y
    cur_y_sample_pts = []
    for cur_face in cur_faces:
      cur_face_verts = [cur_verts[v_idx - 1].astype(np.float32) for v_idx in cur_face] # vertices coordinates of the face
      # cur_face_verts
      
      v_a, v_b, v_c = cur_face_verts[:3]

      # v_a = v_a.astype('np.float32')
      # v_b = v_b.astype('np.float32')
      # v_c = v_c.astype('np.float32')

      v_ab, v_ac = v_b - v_a, v_c - v_a

      v_ab = v_ab / np.minimum(np.sqrt(np.sum(v_ab ** 2)), 1e-9)
      v_ac = v_ac / np.minimum(np.sqrt(np.sum(v_ac ** 2)), 1e-9)

      v_n = np.cross(v_ab, v_ac) # (3,)
      a, b, c = v_n[0].item(), v_n[1].item(), v_n[2].item()
      # a * x + b * y + c * z = 0
      # a * x + c * z = -b * y
      # b_mult_y
      # sampled_y
      b_mult_y = -b * cur_sampled_y # .item() # cur_sampled_y.item()
      # a, b, c
      # 
      # a1 * x + b1 * y + c1 * z = 0
      # Q learning
      # x = (-b * y - c * z) / a
      # b_mult_y
      # (x1, y1, z1), (x2, y2, z2)
      intersected_verts = []
      for i_st in range(len(cur_face_verts)):
        i_ed = (i_st + 1) % len(cur_face_verts) # i_ed
        st_vert, ed_vert = cur_face_verts[i_st], cur_face_verts[i_ed]
        max_vert_y, min_vert_y = st_vert[1].item(), ed_vert[1].item()
        max_vert_y, min_vert_y = max(max_vert_y, min_vert_y), min(max_vert_y, min_vert_y)

        max_vert_x, min_vert_x = st_vert[0].item(), ed_vert[0].item()
        max_vert_x, min_vert_x = max(max_vert_x, min_vert_x), min(max_vert_x, min_vert_x)

        max_vert_z, min_vert_z = st_vert[2].item(), ed_vert[2].item()
        max_vert_z, min_vert_z = max(max_vert_z, min_vert_z), min(max_vert_z, min_vert_z)
        if not (cur_sampled_y >= min_vert_y and cur_sampled_y <= max_vert_y):
          continue
        t_val = (cur_sampled_y - min_vert_y) / max(max_vert_y - min_vert_y, 1e-9)
        cur_x, cur_z = min_vert_x + t_val * (max_vert_x - min_vert_x), min_vert_z + t_val * (max_vert_z - min_vert_z)
        # (cur_x, cur_sampled_y, cur_z)
        intersected_verts.append([cur_x, cur_sampled_y, cur_z])
      intersected_verts = np.array(intersected_verts, dtype=np.float32) # .float() # intersected_verts
      # print(f"length of intersected vertices: {len(intersected_verts)}")

      if intersected_verts.shape[0] == 0:
        continue
      a_vert, b_vert = intersected_verts[0], intersected_verts[1]
      ### 
      ab_dis = b_vert - a_vert
      # ab_len = np.(ab_dis, p=2)
      ab_len = np.sqrt(np.sum(ab_dis ** 2))
      # ab_dis = ab_dis / np.norm(ab_dis, p=2) # tot_unit_vector ---
      ab_dis = ab_dis / np.sqrt(np.sum(ab_dis ** 2))
      cur_interval_sample_pts = []

      # ab_dis: distance between vertices
      nn_cur_interval_samples = int(ab_len)
      for i_s in range(nn_cur_interval_samples): # ab_len 
        cur_interval_cur_sample_pt = a_vert + ab_dis * (i_s) # * ab_len / nn_per_verts_samples
        cur_interval_sample_pts.append(np.reshape(cur_interval_cur_sample_pt, (1, 3)))

      # for i_s in range(nn_per_verts_samples):
      #   cur_interval_cur_sample_pt = a_vert + ab_dis * (i_s) * ab_len / nn_per_verts_samples
      #   cur_interval_sample_pts.append(cur_interval_cur_sample_pt)
      if len(cur_interval_sample_pts) > 0:
        cur_interval_sample_pts = np.concatenate(cur_interval_sample_pts, axis=0) # n_tot_samples x 3
        cur_y_sample_pts.append(cur_interval_sample_pts)

    if len(cur_y_sample_pts) > 0:
      cur_y_sample_pts = np.concatenate(cur_y_sample_pts, axis=0) # tot_n_y_sample x 3
      sampled_pts.append(cur_y_sample_pts)
  
  if len(sampled_pts) > 0:
    sampled_pts = np.concatenate(sampled_pts, axis=0)
  else:
    sampled_pts = np.zeros((0, 3), dtype=np.float32)

  sampled_pts = sampled_pts.astype(np.int32)

  sampled_pts = np.minimum(sampled_pts, 2 ** quantization_bits - 1)
  sampled_pts = np.maximum(sampled_pts, 0)

  tot_res_verts_pts = np.concatenate([cur_verts, sampled_pts], axis=0)

  tot_res_verts_pts_torch = torch.from_numpy(tot_res_verts_pts).float()
  fps_idx = data_utils.farthest_point_sampling(tot_res_verts_pts_torch.unsqueeze(0), n_sampling=min(400, tot_res_verts_pts_torch.size(0)))

  tot_res_verts_pts_torch = tot_res_verts_pts_torch[fps_idx] # sampled vertices
  
  tot_res_verts_pts = tot_res_verts_pts_torch.numpy()
  
  tot_res_verts_pts = data_utils.dequantize_verts(tot_res_verts_pts, quantization_bits)
  cur_verts = data_utils.dequantize_verts(cur_verts, quantization_bits)

  dist_cur_verts_tot_res_verts = np.sum((np.reshape(cur_verts, (cur_verts.shape[0], 1, 3)) - np.reshape(tot_res_verts_pts, (1, tot_res_verts_pts.shape[0], 3)) ) ** 2, axis=-1) # cur_verts.shape[0] x tot_res_verts_pts.shape[0]
  minn_cur_verts_tot_res_verts_idx = np.argmin(dist_cur_verts_tot_res_verts, axis=-1).tolist() # cur_verts.ahepe[0]
  # cur_vert_idx_to_tot_vert_idx: 
  cur_vert_idx_to_tot_vert_idx = {ii + 1: kk + 1 for ii, kk in enumerate(minn_cur_verts_tot_res_verts_idx)}
  tot_res_faces = []
  for cur_face in cur_faces:
    cur_res_face = [cur_vert_idx_to_tot_vert_idx[f_idx] for f_idx in cur_face]
    tot_res_faces.append(cur_res_face)
  cur_faces = tot_res_faces
  return tot_res_verts_pts, cur_faces

def reform_meshes(fr_data_folder, sv_data_folder):
  tot_part_nms = os.listdir(fr_data_folder)
  tot_part_nms = [fn for fn in tot_part_nms if os.path.isdir(os.path.join(fr_data_folder, fn))]
  for cur_part_nm in tot_part_nms:
    print(f" Processing part name: {cur_part_nm}...")
    cur_part_folder = os.path.join(fr_data_folder, cur_part_nm)
    cur_part_folder_sv = os.path.join(sv_data_folder, cur_part_nm)
    os.makedirs(cur_part_folder_sv, exist_ok=True)
    tot_obj_fns = os.listdir(cur_part_folder)
    tot_obj_fns = [fn for fn in tot_obj_fns if fn.endswith(".obj")]
    for obj_fn in tot_obj_fns:
      cur_fr_obj_fn = os.path.join(cur_part_folder, obj_fn)
      cur_sv_obj_fn = os.path.join(cur_part_folder_sv, obj_fn)
      res_verts_pts, faces = get_sampled_points(cur_fr_obj_fn, nn_y_intervals=10, nn_per_verts_samples=10)
      # sv_obj_fn = "tmp.obj"
      data_utils.save_obj_file(res_verts_pts, faces, cur_sv_obj_fn)
  

# # get_sampled_points(obj_fn, nn_y_intervals=10, nn_per_verts_samples=10):
# obj_fn = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized/screwdriver/dof_rootd_Aa001_r/0006.obj"
# res_verts_pts, faces = get_sampled_points(obj_fn, nn_y_intervals=10, nn_per_verts_samples=10)
# sv_obj_fn = "tmp.obj"
# data_utils.save_obj_file(res_verts_pts, faces, sv_obj_fn)

fr_rt_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized"
sv_rt_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_30_normalized_reform_v2"
os.makedirs(sv_rt_folder, exist_ok=True)
tot_motion_cats = os.listdir(fr_rt_folder)

# tot_motion_cats = [fn for fn in tot_motion_cats if os.path.isdir(os.path.join(fr_rt_folder, fn)) and fn != "globe" and fn != "screwdriver"]
sv_tot_motion_cats = os.listdir(sv_rt_folder)

sv_tot_motion_cats = [fn for fn in sv_tot_motion_cats if os.path.isdir(os.path.join(fr_rt_folder, fn))]
tot_motion_cats = [fn for fn in tot_motion_cats if fn not in sv_tot_motion_cats]
for cur_motion_cat in tot_motion_cats:
  print(f"Processing motion category: {cur_motion_cat}")
  cur_motion_cat_sv = os.path.join(sv_rt_folder, cur_motion_cat)
  os.makedirs(cur_motion_cat_sv, exist_ok=True)
  cur_motion_cat_fr = os.path.join(fr_rt_folder, cur_motion_cat)
  reform_meshes(cur_motion_cat_fr, cur_motion_cat_sv)



Processing motion category: globe
 Processing part name: dof_rootd_Ba001_r...


/home/xueyi/anaconda3/envs/mage/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in true_divide
/home/xueyi/anaconda3/envs/mage/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide
/home/xueyi/anaconda3/envs/mage/lib/python3.6/site-packages/ipykernel_launcher.py:91: RuntimeWarning: invalid value encountered in true_divide


 Processing part name: dof_rootd_Aa001_r...
 Processing part name: none_motion...
 Processing part name: dof_Ba001_Aa001_r...
 Processing part name: dof_rootd_Ca001_r...
Processing motion category: screwdriver
 Processing part name: dof_rootd_Aa001_r...
 Processing part name: none_motion...
Processing motion category: bike
 Processing part name: dof_rootd_Ca002_r...
 Processing part name: dof_rootd_Ba001_r...
 Processing part name: dof_rootd_Ca003_r...
 Processing part name: dof_rootd_Aa001_r...
 Processing part name: dof_Aa001_Ca001_r...
 Processing part name: none_motion...
 Processing part name: dof_rootd_Ba002_r...
Processing motion category: wine_bottle
 Processing part name: dof_rootd_Ba001_t...
 Processing part name: dof_rootd_Aa001_t...
 Processing part name: none_motion...
Processing motion category: watch
 Processing part name: dof_rootd_Aa003_r...
 Processing part name: dof_rootd_Aa001_r...


IndexError: index 1 is out of bounds for axis 0 with size 0

In [5]:
def trimesh_voxilization(fr_cat_folder, sv_cat_folder):
    import trimesh
    from trimesh.voxel import creation
    pitch = 0.01

    insts = os.listdir(fr_cat_folder)
    insts = [fn for fn in insts if os.path.isdir(os.path.join(fr_cat_folder, fn))]
    for inst in insts:
        sv_inst_fn = os.path.join(sv_cat_folder, inst)
        os.makedirs(sv_inst_fn, exist_ok=True)
        fr_inst_fn = os.path.join(fr_cat_folder, inst)
        objs = os.listdir(fr_inst_fn)
        objs = [fn for fn in objs if fn.endswith(".obj")]
        for obj_fn in objs:
            cur_obj_fn = os.path.join(fr_inst_fn, obj_fn)
            cur_obj_sv_fn = os.path.join(sv_inst_fn, obj_fn)
            cur_verts, cur_faces = read_obj_file_ours(cur_obj_fn, minus_one=True)
            cur_verts = data_utils.center_vertices(cur_verts)
            cur_verts = data_utils.normalize_vertices_scale(cur_verts)
            cur_mesh = trimesh.Trimesh(vertices=cur_verts, faces=cur_faces)
            # voxels = creation.voxelize(cur_mesh, pitch=0.005, method='subdivide')
            voxels = creation.voxelize(cur_mesh, pitch=pitch, method='subdivide')
            tot_pts = []
            n_x, n_y, n_z = voxels.shape
            for i_x in range(n_x):
                for i_y in range(n_y):
                    for i_z in range(n_z):
                        cur_voxel_val = voxels.matrix[i_x, i_y, i_z].item()
                        if cur_voxel_val > 0.1:
                            # print(i_x, i_y, i_z, cur_voxel_val)
                            tot_pts.append([i_x, i_y, i_z])

            tot_pts = np.array(tot_pts, dtype=np.float32)
            tot_pts = data_utils.center_vertices(tot_pts)
            tot_pts = data_utils.normalize_vertices_scale(tot_pts)
            
            dist_cur_verts_tot_res_verts = np.sum((np.reshape(cur_verts, (cur_verts.shape[0], 1, 3)) - np.reshape(tot_pts, (1, tot_pts.shape[0], 3)) ) ** 2, axis=-1) # cur_verts.shape[0] x tot_res_verts_pts.shape[0]
            minn_cur_verts_tot_res_verts_idx = np.argmin(dist_cur_verts_tot_res_verts, axis=-1).tolist() # cur_verts.ahepe[0]
            # cur_vert_idx_to_tot_vert_idx: 
            cur_vert_idx_to_tot_vert_idx = {ii: kk  for ii, kk in enumerate(minn_cur_verts_tot_res_verts_idx)}
            tot_res_faces = []
            for cur_face in cur_faces:
                cur_res_face = [cur_vert_idx_to_tot_vert_idx[f_idx] for f_idx in cur_face]
                tot_res_faces.append(cur_res_face)
            cur_faces = tot_res_faces
            data_utils.save_obj_file(tot_pts, cur_faces, cur_obj_sv_fn, add_one=True)
        
            

fr_rt_folder = "/nas/datasets/gen/datasets/MotionDataset_processed"
sv_rt_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_0.01"
os.makedirs(sv_rt_folder, exist_ok=True)
tot_motion_cats = os.listdir(fr_rt_folder)

# tot_motion_cats = [fn for fn in tot_motion_cats if os.path.isdir(os.path.join(fr_rt_folder, fn)) and fn != "globe" and fn != "screwdriver"]
sv_tot_motion_cats = os.listdir(sv_rt_folder)

sv_tot_motion_cats = [fn for fn in sv_tot_motion_cats if os.path.isdir(os.path.join(fr_rt_folder, fn))]
# tot_motion_cats = [fn for fn in tot_motion_cats if fn not in sv_tot_motion_cats]
for cur_motion_cat in tot_motion_cats:
  print(f"Processing motion category: {cur_motion_cat}")
  cur_motion_cat_sv = os.path.join(sv_rt_folder, cur_motion_cat)
  os.makedirs(cur_motion_cat_sv, exist_ok=True)
  cur_motion_cat_fr = os.path.join(fr_rt_folder, cur_motion_cat)
  trimesh_voxilization(cur_motion_cat_fr, cur_motion_cat_sv)


Processing motion category: globe


KeyboardInterrupt: 

In [2]:


def get_binvox_data(root="/share/xueyi/proj_data/Motion_Aligned_2", sv_root="", shape_type="oven", shape_idxes=None):
    root = os.path.join(root, shape_type)
    shape_idxes = os.listdir(root) 
    shape_idxes = sorted(shape_idxes)
    # shape_idxes = [tmpp for tmpp in shape_idxes if tmpp[0] != "."]
    shape_idxes = [fn for fn in shape_idxes if os.path.isdir(os.path.join(root, fn))]

    sv_root = os.path.join(sv_root, shape_type)
    os.makedirs(sv_root, exist_ok=True) # make motion category folder
    
    for shp_idx in shape_idxes:
      cur_shape_folder = os.path.join(root, shp_idx)
      cur_shape_sv_folder = os.path.join(sv_root, shp_idx)
      os.makedirs(cur_shape_sv_folder, exist_ok=True) # make shape folder

      cur_shape_part_folder = cur_shape_folder
      cur_shape_part_sv_folder = cur_shape_sv_folder
      
      # cur_parts = os.listdir(cur_shape_folder)
      # cur_parts = [fn for fn in cur_parts if os.path.isdir(os.path.join(cur_shape_folder, fn))]
      # for cur_part in cur_parts:
      # cur_shape_part_folder = os.path.join(cur_shape_folder, cur_part)
      # cur_shape_part_sv_folder = os.path.join(cur_shape_sv_folder, cur_part)
      # os.makedirs(cur_shape_part_sv_folder, exist_ok=True) # make part folder
      
      obj_files = os.listdir(cur_shape_part_folder)
      obj_files = [fn for fn in obj_files if fn.endswith(".obj")]
      
      for obj_fn in obj_files:
        cur_obj_fn = os.path.join(cur_shape_part_folder, obj_fn)
        cur_obj_sv_fn = os.path.join(cur_shape_part_sv_folder, obj_fn)

        os.system(f"cp {cur_obj_fn} {cur_obj_sv_fn}")
        os.system(f"/home/xueyi/gen/cuda_voxelizer/build/cuda_voxelizer -f {cur_obj_sv_fn} -s 64")

    # shape_idxes = 

# save_folder = "/share/xueyi/proj_data/Motion_Samples"
# shape_type = "eyeglasses"
root = "/nas/datasets/gen/datasets/MotionDataset_processed"
sv_root = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64"
os.makedirs(sv_root, exist_ok=True)
motion_cats = os.listdir(root) # root
motion_cats = [fn for fn in motion_cats if os.path.isdir(os.path.join(root, fn))]
motion_sv_cats = os.listdir(sv_root)
motion_sv_cats = [fn for fn in motion_sv_cats if os.path.isdir(os.path.join(sv_root, fn))] # is dir ---> 
# motion_cats = [fn for fn in motion_cats if fn not in motion_sv_cats]

shape_idxes = None
for motion_cat in motion_cats:
  print(f"processing motion cat: {motion_cat}")
  get_binvox_data(root, sv_root=sv_root, shape_type=motion_cat, shape_idxes=shape_idxes) # binvox data for...



In [5]:


import binvox_rw


def get_statistics_info(dataset_root_folder):
  motion_cats = os.listdir(dataset_root_folder)
  motion_cats = [fn for fn in motion_cats if os.path.isdir(os.path.join(dataset_root_folder, fn))]
  motion_cat_to_infos = {}
  for cat in motion_cats:
    print(f"process category: {cat}")
    cur_cat_folder = os.path.join(dataset_root_folder, cat)
    cur_cat_insts = os.listdir(cur_cat_folder)
    cur_cat_insts = [fn for fn in cur_cat_insts if os.path.isdir(os.path.join(cur_cat_folder, fn))]
    inst_idx_to_fn_to_pts_nn = {}
    for cur_inst in cur_cat_insts: # inst_fn
      print(f"process instance: {cur_inst}")
      cur_cat_cur_inst_folder = os.path.join(cur_cat_folder, cur_inst)
      cur_cat_cur_insts = os.listdir(cur_cat_cur_inst_folder)
      cur_cat_cur_insts = [fn for fn in cur_cat_cur_insts if fn.endswith(".binvox")]
      fn_to_pts_nn = {}
      for cur_cat_cur_inst_fn in cur_cat_cur_insts:
        cur_inst_fn = os.path.join(cur_cat_cur_inst_folder, cur_cat_cur_inst_fn)
        # import binvox_rw
        with open(cur_inst_fn, 'rb') as f:
            model = binvox_rw.read_as_3d_array(f)
        pts = []
        dim_x, dim_y, dim_z = model.dims

        for i_x in range(dim_x):
            for i_y in range(dim_y):
                for i_z in range(dim_z):
                    cur_model_data = int(model.data[i_x, i_y, i_z].item())
                    if cur_model_data > 0:
                        # print(cur_model_data)
                        pts.append([i_x, i_y, i_z])
        cur_inst_pts_nn = len(pts)
        fn_to_pts_nn[cur_inst_fn] = cur_inst_pts_nn
        # cur_inst_split_fns = cur_cat_cur_inst_fn.split(".")[0].split("_") # split(".") --> [...] + [obj_64] + [binvox]
        # cur_inst_scale_idx = int(cur_inst_split_fns[-1])
        # cur_inst_part_fn = "_".join(cur_inst_scale_idx[:-2]) # [...] + [s] + [scale_idx]
        # cur_inst_part_fn
        # cur_inst_part_fn
      inst_idx_to_fn_to_pts_nn[cur_inst] = fn_to_pts_nn
    
    sv_meta_info_fn = os.path.join(cur_cat_folder, "meta_infos.npy")
    np.save(sv_meta_info_fn, inst_idx_to_fn_to_pts_nn)
    
    motion_cat_to_infos[cat] = inst_idx_to_fn_to_pts_nn
  print(motion_cat_to_infos['eyeglasses'])
  np.save("meta_infos.npy", motion_cat_to_infos)


dataset_root_folder = "/nas/datasets/gen/datasets/MotionDataset_processed_voxelized_64_v9_ns_50" 
get_statistics_info(dataset_root_folder)
        
        
      

process category: globe
process instance: 0001
process category: screwdriver
process instance: 0070
process instance: 0057
process instance: 0065
process instance: 0063
process instance: 0030
process instance: 0001
process instance: 0046
process instance: 0067
process instance: 0041
process instance: 0038
process instance: 0031
process instance: 0068
process instance: 0032
process instance: 0022
process instance: 0043
process instance: 0002
process instance: 0023
process instance: 0048
process instance: 0060
process instance: 0033
process instance: 0020
process instance: 0003
process instance: 0008
process instance: 0025
process instance: 0069
process instance: 0028
process instance: 0059
process instance: 0049
process instance: 0037
process instance: 0052
process instance: 0044
process instance: 0051
process instance: 0053
process instance: 0035
process instance: 0066
process instance: 0013
process instance: 0034
process instance: 0006
process instance: 0021
process instance: 0026
pro

In [6]:

import numpy as np

def get_meta_info(shp_folder):
  tot_cats = os.listdir(shp_folder)
  tot_cats = [fn for fn in tot_cats if os.path.isdir(os.path.join(shp_folder, fn))]
  tot_shp_nn = 0
  for cat in tot_cats:
    cur_cat_folder = os.path.join(shp_folder, cat)
    # meta_info_fn = "meta_infos.npy"
    meta_info_fn = "surface-meta_infos.npy"
    cur_cat_meta_info_fn = os.path.join(cur_cat_folder, meta_info_fn)
    cur_cat_meta_info = np.load(cur_cat_meta_info_fn, allow_pickle=True).item()
    print(f"length of current category's ({cat}) meta info: {len(cur_cat_meta_info)}.")
    tot_shp_nn += len(cur_cat_meta_info)
    
  print(f"tot_shp_nn: {tot_shp_nn}")

def get_meta_info_category(shp_folder, shp_cat):
  meta_info_fn = "meta_infos.npy"
  shp_cat_folder = os.path.join(shp_folder, shp_cat)
  cur_shp_cat_meta_info_fn = os.path.join(shp_cat_folder, meta_info_fn)
  cur_shp_cat_meta_info = np.load(cur_shp_cat_meta_info_fn, allow_pickle=True).item()
  threshold = 3000
  valid_inst_fns = []
  for cur_inst_fn in cur_shp_cat_meta_info:
    cur_inst_nn_pts = cur_shp_cat_meta_info[cur_inst_fn]
    for vox_fn in cur_inst_nn_pts:
      cur_vox_nn_pts = cur_inst_nn_pts[vox_fn]
      if cur_vox_nn_pts <= threshold:
        valid_inst_fns.append(vox_fn)
    # if cur_inst_nn_pts <= threshold:
      # valid_inst_fns.append(cur_inst_fn)
  print(valid_inst_fns[:5])
  # for inst_fn in valid_inst_fns:
  #   cur_inst_fn = os.path.join(shp_cat_folder, inst_fn)
    

shp_folder = "/nas/datasets/gen/datasets/ShapeNetCore.v2"
# get_meta_info(shp_folder=shp_folder)

cat_id = "03001627"
get_meta_info_category(shp_folder, cat_id)


['/nas/datasets/gen/datasets/ShapeNetCore.v2/03001627/ddfe96c6ec86b8752cbb5ed9636a4451/models/model_normalized.solid.binvox', '/nas/datasets/gen/datasets/ShapeNetCore.v2/03001627/40d202afdcc49c6d35836c728d324152/models/model_normalized.solid.binvox', '/nas/datasets/gen/datasets/ShapeNetCore.v2/03001627/24fdf6e5090bc3442370db05a58bf9c6/models/model_normalized.solid.binvox', '/nas/datasets/gen/datasets/ShapeNetCore.v2/03001627/108b9cb292fd811cf51f77a6d7299806/models/model_normalized.solid.binvox']


In [1]:

import numpy as np
from data_utils import *



def get_connected_mesh(verts, faces):
    old_idx_to_new_idx = {}
    vert_nn = 0
    selected_verts = []
    new_faces = []
    for i_f, cur_f in enumerate(faces):
        cur_new_f = []
        for cur_v in cur_f:
            if cur_v not in old_idx_to_new_idx:
                old_idx_to_new_idx[cur_v] = vert_nn + 1 #### vert_nn + 1
                vert_nn += 1
                selected_verts.append(cur_v)
            cur_new_f.append(old_idx_to_new_idx[cur_v])
        new_faces.append(cur_new_f)

    selected_verts = np.array(selected_verts, dtype=np.long)
    selected_verts = verts[selected_verts - 1] ##### sv x 3
    return selected_verts, new_faces


def reroder_vertices_faces(vertices, faces, sv_obj_fn=""):

  vertices, faces = get_connected_mesh(vertices, faces)
  refered_verts_values = vertices

  # new_idx_to_old_idx = {ii: refered_verts_indices[ii] for ii in range(len(refered_verts_indices))}
  # refered vertices value list
  refered_verts_values_list = [(vertices[ii][0].item(), vertices[ii][1].item(), vertices[ii][2].item()) for ii in range(vertices.shape[0]) ]
  refered_verts_values_arr = np.array(refered_verts_values_list, dtype=[('x', float), ('y', float), ('z', float)])

  sorted_idxes_value = np.argsort(refered_verts_values_arr, order=['y', 'z', 'x'])
  refered_verts_values = refered_verts_values[sorted_idxes_value]
  nn_new_idx_to_old_idx = {}
  for ii_v in range(sorted_idxes_value.shape[0]):
    cur_vert_idx = int(sorted_idxes_value[ii_v].item())
    # cur_vert_old_idx = new_idx_to_old_idx[cur_vert_idx]
    nn_new_idx_to_old_idx[ii_v] = cur_vert_idx
  nn_old_idx_to_new_idx = {nn_new_idx_to_old_idx[kk]: kk for kk in nn_new_idx_to_old_idx} # old idx to new idx
  nn_faces = []
  for cur_face in faces:
    nn_cur_face = []
    for cur_face_idx in cur_face:
      nn_cur_face_idx = nn_old_idx_to_new_idx[cur_face_idx - 1]
      nn_cur_face.append(nn_cur_face_idx + 1) # nn_cur_face ---- vertices indexes of faces should be larger than 0!
    nn_faces.append(nn_cur_face)
  # nn_faces! # nn_faces
  nn_faces = sorted(nn_faces, key=lambda ii_face: min(ii_face), reverse=False)
  permuted_nn_faces = [] # permuted_nn_faces! # permuted_nn_faces...
  for nn_cur_face in nn_faces:
    min_face_idx = np.argmin(np.array(nn_cur_face), axis=0).item()
    min_face_idx = int(min_face_idx)
    permuted_cur_face = nn_cur_face[min_face_idx: ] + nn_cur_face[:min_face_idx]
    permuted_nn_faces.append(permuted_cur_face)
  save_obj_file(refered_verts_values, permuted_nn_faces, sv_obj_fn)

# obj_fn = "/home/xueyi/gen/surface_multigrid_code/meshes/bunny.obj"
# vertices, faces = read_obj_file_ours(obj_fn=obj_fn)
# sv_obj_fn = "/home/xueyi/gen/surface_multigrid_code/meshes/bunny_reordered.obj"
# reroder_vertices_faces(vertices, faces, sv_obj_fn)

#### for partnet ####
# dataset_root_path = "/nas/datasets/gen/datasets/PartNet/data_v0_deci_0.2"
# inst_nms = os.listdir(dataset_root_path)
# inst_nms = [fn for fn in inst_nms if os.path.isdir(os.path.join(dataset_root_path, fn))]
# debug = True
# debug = False
# # inst_nms =["347"]
# for inst_nm in inst_nms:
#   print(f"processing {inst_nm}")
#   cur_inst_folder = os.path.join(dataset_root_path, inst_nm)
#   cur_inst_folder = os.path.join(cur_inst_folder, "objs")
#   cur_inst_objs = os.listdir(cur_inst_folder)
#   cur_inst_objs = [fn for fn in cur_inst_objs if fn.endswith(".obj")]
#   for obj_fn in cur_inst_objs:
#     cur_obj_fn = os.path.join(cur_inst_folder, obj_fn)
#     vertices, faces = read_obj_file_ours(obj_fn=cur_obj_fn)
#     reroder_vertices_faces(vertices, faces, cur_obj_fn)
#   if debug:
#     break

#### prserved edge from 10 to 100?

# #### partfirst dataset ####
# dataset_root_path = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_0.2_normalized"
# inst_nms = os.listdir(dataset_root_path)
# inst_nms = [fn for fn in inst_nms if os.path.isdir(os.path.join(dataset_root_path, fn))]
# debug = True
# # debug = False
# # inst_nms =["347"]
# inst_nms = ["eyeglasses"]
# for inst_nm in inst_nms:
#   print(f"processing {inst_nm}")
  
#   cur_inst_folder = os.path.join(dataset_root_path, inst_nm)

#   part_nms = os.listdir(cur_inst_folder)
#   part_nms = [fn for fn in part_nms if os.path.isdir(os.path.join(cur_inst_folder, fn))]
#   for part_nm in part_nms:
#     cur_part_folder = os.path.join(cur_inst_folder, part_nm)
#     cur_part_objs = os.listdir(cur_part_folder)
#     cur_part_objs = [fn for fn in cur_part_objs if fn.endswith(".obj")]
#     for obj_fn in cur_part_objs:
#       cur_obj_fn = os.path.join(cur_part_folder, obj_fn)
#       vertices, faces = read_obj_file_ours(obj_fn=cur_obj_fn)
#       reroder_vertices_faces(vertices, faces, cur_obj_fn)
#   # cur_inst_folder = os.path.join(cur_inst_folder, "objs")
#   # cur_inst_objs = os.listdir(cur_inst_folder)
#   # cur_inst_objs = [fn for fn in cur_inst_objs if fn.endswith(".obj")]
#   # for obj_fn in cur_inst_objs:
#   #   cur_obj_fn = os.path.join(cur_inst_folder, obj_fn)
#   #   vertices, faces = read_obj_file_ours(obj_fn=cur_obj_fn)
#   #   reroder_vertices_faces(vertices, faces, cur_obj_fn)
#   # if debug:
#   #   break
# #### partfirst dataset ####

# dataset_root_path = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_0.2_normalized"
# dataset_root_path = "/nas/datasets/gen/datasets/MotionDataset_processed_collapsed_sel"
# motion_cats = ['eyeglasses']

# for cur_motion_cat in motion_cats:
#   cur_motion_root_path = os.path.join(dataset_root_path, cur_motion_cat)
#   inst_fns = os.listdir(cur_motion_root_path)
#   inst_fns = [fn for fn in inst_fns if os.path.isdir(os.path.join(cur_motion_root_path, fn))]
#   for inst_fn in inst_fns:
#     cur_inst_folder = os.path.join(cur_motion_root_path, inst_fn)
#     objs = os.listdir(cur_inst_folder)
#     objs = [fn for fn in objs if fn.endswith(".obj")]
#     for obj_fn in objs:
#       cur_obj_fn = os.path.join(cur_inst_folder, obj_fn)
#       vertices, faces = read_obj_file_ours(cur_obj_fn)
#       reroder_vertices_faces(vertices, faces, cur_obj_fn)



# # dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh"
# dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh"
# # motion_cats = ['screw_1']
# motion_cats = ["screw_1_planar"]

# for cur_motion_cat in motion_cats:
#   cur_motion_root_path = os.path.join(dataset_root_path, cur_motion_cat)
#   inst_fns = os.listdir(cur_motion_root_path)
#   inst_fns = [fn for fn in inst_fns if os.path.isdir(os.path.join(cur_motion_root_path, fn))]
#   for inst_fn in inst_fns:
#     cur_inst_folder = os.path.join(cur_motion_root_path, inst_fn)
#     objs = os.listdir(cur_inst_folder)
#     objs = [fn for fn in objs if fn.endswith(".obj")]
#     for obj_fn in objs:
#       cur_obj_fn = os.path.join(cur_inst_folder, obj_fn)
#       vertices, faces = read_obj_file_ours(cur_obj_fn)
#       reroder_vertices_faces(vertices, faces, cur_obj_fn)

# dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh"
dataset_root_path = "/nas/datasets/gen/datasets/MotionDataset_processed_2"
dataset_root_path = "/nas/datasets/gen/datasets/bsp_abstraction_3_processed"
# motion_cats = ['screw_1']
motion_cats = ["lamp"]
motion_cats = ["eyeglasses"]
motion_cats = ["pen"]
motion_cats = ["eyeglasses"]

for cur_motion_cat in motion_cats:
  cur_motion_root_path = os.path.join(dataset_root_path, cur_motion_cat)
  inst_fns = os.listdir(cur_motion_root_path)
  inst_fns = [fn for fn in inst_fns if os.path.isdir(os.path.join(cur_motion_root_path, fn))]
  for inst_fn in inst_fns:
    cur_inst_folder = os.path.join(cur_motion_root_path, inst_fn)
    objs = os.listdir(cur_inst_folder)
    objs = [fn for fn in objs if fn.endswith(".obj")]
    for obj_fn in objs:
      cur_obj_fn = os.path.join(cur_inst_folder, obj_fn)
      vertices, faces = read_obj_file_ours(cur_obj_fn)
      reroder_vertices_faces(vertices, faces, cur_obj_fn)

# inst_nms = os.listdir(dataset_root_path)
# inst_nms = [fn for fn in inst_nms if os.path.isdir(os.path.join(dataset_root_path, fn))]
# debug = True
# # debug = False
# # inst_nms =["347"]
# inst_nms = ["eyeglasses"]
# for inst_nm in inst_nms:
#   print(f"processing {inst_nm}")

#   cur_inst_folder = os.path.join(dataset_root_path, inst_nm)

#   part_nms = os.listdir(cur_inst_folder)
#   part_nms = [fn for fn in part_nms if os.path.isdir(os.path.join(cur_inst_folder, fn))]
#   for part_nm in part_nms:
#     cur_part_folder = os.path.join(cur_inst_folder, part_nm)
#     cur_part_objs = os.listdir(cur_part_folder)
#     cur_part_objs = [fn for fn in cur_part_objs if fn.endswith(".obj")]
#     for obj_fn in cur_part_objs:
#       cur_obj_fn = os.path.join(cur_part_folder, obj_fn)
#       vertices, faces = read_obj_file_ours(obj_fn=cur_obj_fn)
#       reroder_vertices_faces(vertices, faces, cur_obj_fn)

def read_mesh_with_mtl(mesh_fn, minus_one=False):
    vertices = []
    faces = []
    cur_faces = []
    tot_faces = []
    mlt_idx_to_faces = {}
    cur_mlt_idx = 0
    print(f"cur_mesh_fn: {mesh_fn}")
    with open(mesh_fn, "r") as rf:
        for line in rf:
            cur_splits = line.strip().split(" ")
            if len(cur_splits) <= 0:
                continue
            cur_indicator = cur_splits[0]
            # print(f"cur_indicator: {cur_indicator}")
            if cur_indicator == "usemtl":
                if len(cur_faces) > 0:
                  mlt_idx_to_faces[cur_mlt_idx] = cur_faces
                  faces.append(cur_faces)
                  tot_faces += cur_faces
                  cur_faces = []
                cur_mlt_idx = int(cur_splits[1][1:])
            elif cur_indicator == "v":
                cur_verts = cur_splits[1:]
                cur_verts = [float(vv) for vv in cur_verts]
                cur_verts = np.array(cur_verts, dtype=np.float)
                vertices.append(cur_verts)
            elif cur_indicator == "f":
                cur_face_indices = cur_splits[1:]
                cur_face_indices = [int(ff) - 1 if minus_one else int(ff) for ff in cur_face_indices]
                cur_faces.append(cur_face_indices)
            else:
                continue
        rf.close()
    if len(cur_faces) > 0:
        mlt_idx_to_faces[cur_mlt_idx] = cur_faces
        faces.append(cur_faces)
        tot_faces += cur_faces
    vertices = np.array(vertices)
    print(f"Mesh loaded with vertices: {vertices.shape}")
    return vertices, faces, tot_faces, mlt_idx_to_faces

def get_sub_mesh(vertices, faces, sub_faces):
  selected_idxes = []
  old_idx_to_new_idx = {}
  new_faces = []
  for i_f, cur_f in enumerate(sub_faces):
    cur_new_face = []
    for cur_v in cur_f:
      if cur_v not in old_idx_to_new_idx:
        old_idx_to_new_idx[cur_v] = len(old_idx_to_new_idx) + 1 ### new idxes
        selected_idxes.append(cur_v - 1)
      cur_new_face.append(old_idx_to_new_idx[cur_v]) #### old_idx_to_new_idx ####
    new_faces.append(cur_new_face)
  selected_idxes = np.array(selected_idxes, dtype=np.long)
  selected_vertices = vertices[selected_idxes]
  return selected_vertices, new_faces


def reform_mtl_obj_file(obj_fn):
  vertices, faces, tot_faces, mlt_idx_to_faces = read_mesh_with_mtl(obj_fn, minus_one=False)
  mtl_idx_to_verts_faces = {}
  for cur_mlt_idx in mlt_idx_to_faces:
    cur_cluster_faces = mlt_idx_to_faces[cur_mlt_idx]
    cur_cluster_vertices, cur_cluster_faces = get_sub_mesh(vertices, tot_faces, cur_cluster_faces)
    mtl_idx_to_verts_faces[cur_mlt_idx] = (cur_cluster_vertices, cur_cluster_faces)
  return mtl_idx_to_verts_faces

fr_mesh_folder = "/home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_32"
fr_mesh_folder = "/home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_64"
sv_mesh_folder = "/nas/datasets/gen/datasets/bsp_abstraction_2"
sv_mesh_folder = "/nas/datasets/gen/datasets/bsp_abstraction_3"
sv_mesh_folder = "/nas/datasets/gen/datasets/bsp_abstraction_4"
sv_mesh_folder = "/nas/datasets/gen/datasets/bsp_abstraction_5"
os.makedirs(sv_mesh_folder, exist_ok=True)
motion_cat = "eyeglasses"
sv_mesh_folder = os.path.join(sv_mesh_folder, motion_cat)
os.makedirs(sv_mesh_folder, exist_ok=True)
sv_motion_cat_mesh_folder = sv_mesh_folder

# level = "summary"
# sv_mesh_folder = os.path.join(sv_mesh_folder, level)
# os.makedirs(sv_mesh_folder, exist_ok=True)


tot_n_meshes = 43
for i_mesh in range(tot_n_meshes):
  cur_mesh_fn = f"{i_mesh}_bsp.obj"
  cur_mesh_fr_fn = os.path.join(fr_mesh_folder, cur_mesh_fn)
  mtl_idx_to_verts_faces = reform_mtl_obj_file(cur_mesh_fr_fn)
  for mlt_idx in mtl_idx_to_verts_faces:
    cur_mtl_vertices, cur_mtl_faces = mtl_idx_to_verts_faces[mlt_idx]
    cur_mlt_idx_sv_folder = os.path.join(sv_motion_cat_mesh_folder, f"cvx_{mlt_idx}")
    os.makedirs(cur_mlt_idx_sv_folder, exist_ok=True)
    cur_obj_cur_mlt_idx_sv_fn = os.path.join(cur_mlt_idx_sv_folder, cur_mesh_fn)
    ##### save the current mtl's obj file to corresponding file #####
    save_obj_file(cur_mtl_vertices, cur_mtl_faces, obj_fn=cur_obj_cur_mlt_idx_sv_fn)



# tot_n_meshes = 43
# for i_mesh in range(tot_n_meshes):
#   cur_mesh_fn = f"{i_mesh}_bsp.obj"
#   cur_mesh_fr_fn = os.path.join(fr_mesh_folder, cur_mesh_fn)
#   verts, faces = read_obj_file_ours(cur_mesh_fr_fn, minus_one=False)
#   cur_mesh_sv_fn = os.path.join(sv_mesh_folder, cur_mesh_fn)
#   print(f"processing mesh {cur_mesh_fn} from {cur_mesh_fr_fn} to {cur_mesh_sv_fn}.")
#   # save_obj_file(verts, faces, obj_fn=cur_mesh_sv_fn)

#   reroder_vertices_faces(verts, faces, sv_obj_fn=cur_mesh_sv_fn)
  


/home/xueyi/anaconda3/envs/mage/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cur_mesh_fn: /home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_64/0_bsp.obj
Mesh loaded with vertices: (102, 3)
cur_mesh_fn: /home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_64/1_bsp.obj
Mesh loaded with vertices: (102, 3)
cur_mesh_fn: /home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_64/2_bsp.obj
Mesh loaded with vertices: (90, 3)
cur_mesh_fn: /home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_64/3_bsp.obj
Mesh loaded with vertices: (198, 3)
cur_mesh_fn: /home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_64/4_bsp.obj
Mesh loaded with vertices: (192, 3)
cur_mesh_fn: /home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_64/5_bsp.obj
Mesh loaded with vertices: (30, 3)
cur_mesh_fn: /home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_64/6_bsp.obj
Mesh loaded with vertices: (108, 3)
cur_mesh_fn: /home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_64/7_bsp.obj
Mesh loaded with vertices: (174, 3)
cur_mesh_fn: /home/xueyi/gen/BSP-NET-pytorch/samples/bsp_ae_out_64/8_bsp.obj
Mesh loaded with vert

In [4]:

import numpy as np
from data_utils import read_obj_file_ours

def flatten_faces(faces):
  """Converts from list of faces to flat face array with stopping indices."""
  if not faces:
    return np.array([0])
  else:
    l = [f + [-1] for f in faces[:-1]]
    l += [faces[-1] + [-2]]
    return np.array([item for sublist in l for item in sublist]) + 2  # pylint: disable=g-complex-comprehension


def get_pure_foler_meta_info(dataset_root_path):
  meta_info_fn = os.path.join(dataset_root_path, "meta_infos.npy")
  # if os.path.exists(meta_info_fn):
  #   obj_fn_to_stats = np.load(meta_info_fn, allow_pickle=True).item()
  # else:
  #   obj_fn_to_stats = {}
  obj_fn_to_stats = {}
  obj_fns = os.listdir(dataset_root_path)
  obj_fns = [fn for fn in obj_fns if fn.endswith(".obj")]
  for obj_fn in obj_fns:
    cur_obj_fn = os.path.join(dataset_root_path, obj_fn)
    if cur_obj_fn in obj_fn_to_stats:
      continue
    verts, faces = read_obj_file_ours(cur_obj_fn)
    flat_faces = flatten_faces(faces)
    # flatten_faces 
    nn_verts, nn_faces = verts.shape[0], len(faces) * 3
    nn_faces = flat_faces.shape[0]
    obj_fn_to_stats[cur_obj_fn] = (nn_verts, nn_faces)
  np.save(meta_info_fn, obj_fn_to_stats)
  print(f"Meta-infos saved to {meta_info_fn}!")


def get_motion_dataset_meta_info(dataset_root_path):
  meta_info_fn = os.path.join(dataset_root_path, "meta_infos.npy")
  if os.path.exists(meta_info_fn):
    obj_fn_to_stats = np.load(meta_info_fn, allow_pickle=True).item()
  else:
    obj_fn_to_stats = {}

  inst_fns = os.listdir(dataset_root_path)
  inst_fns = [fn for fn in inst_fns if os.path.isdir(os.path.join(dataset_root_path, fn))]
  # obj_fn_to_stats = {}
  for fn in inst_fns:
    
    cur_inst_folder = os.path.join(dataset_root_path, fn)
    # cur_inst_folder = os.path.join(cur_inst_folder, "objs")
  
    tot_objs = os.listdir(cur_inst_folder)
    tot_objs = [fn for fn in tot_objs if fn.endswith(".obj")]
    for cur_obj in tot_objs:
      cur_obj_fn = os.path.join(cur_inst_folder, cur_obj)
      if cur_obj_fn in obj_fn_to_stats:
        continue
      verts, faces = read_obj_file_ours(cur_obj_fn)
      nn_verts, nn_faces = verts.shape[0], len(faces) * 3
      obj_fn_to_stats[cur_obj_fn] = (nn_verts, nn_faces)
  # meta_info_fn = os.path.join(dataset_root_path, "meta_infos.npy")
  np.save(meta_info_fn, obj_fn_to_stats)
  print(f"Meta-infos saved to {meta_info_fn}!")


def get_partnet_meta_info(dataset_root_path):
  meta_info_fn = os.path.join(dataset_root_path, "meta_infos.npy")
  # if os.path.exists(meta_info_fn):
  #   obj_fn_to_stats = np.load(meta_info_fn, allow_pickle=True).item()
  # else:
  #   obj_fn_to_stats = {}
  obj_fn_to_stats = {}
  inst_fns = os.listdir(dataset_root_path)
  inst_fns = [fn for fn in inst_fns if os.path.isdir(os.path.join(dataset_root_path, fn))]
  # obj_fn_to_stats = {}
  for fn in inst_fns:
    
    cur_inst_folder = os.path.join(dataset_root_path, fn)
    cur_inst_folder = os.path.join(cur_inst_folder, "objs")
    tot_objs = os.listdir(cur_inst_folder)
    tot_objs = [fn for fn in tot_objs if fn.endswith(".obj")]
    for cur_obj in tot_objs:
      cur_obj_fn = os.path.join(cur_inst_folder, cur_obj)
      if cur_obj_fn in obj_fn_to_stats:
        continue
      verts, faces = read_obj_file_ours(cur_obj_fn)
      nn_verts, nn_faces = verts.shape[0], len(faces) * 3
      obj_fn_to_stats[cur_obj_fn] = (nn_verts, nn_faces)
  # meta_info_fn = os.path.join(dataset_root_path, "meta_infos.npy")
  np.save(meta_info_fn, obj_fn_to_stats)
  print(f"Meta-infos saved to {meta_info_fn}!")

#### subd_dataset_meta_ifno
def get_subd_dataset_meta_info(dataset_root_path):
  tot_insts = os.listdir(dataset_root_path)
  tot_insts = [fn for fn in tot_insts if os.path.isdir(os.path.join(dataset_root_path, fn))]
  inst_fn_to_inst_info = {}
  for cur_inst in tot_insts:
    print("Processing instances:", cur_inst)
    cur_inst_info_dict = {}
    cur_inst_folder = os.path.join(dataset_root_path, cur_inst)
    cur_inst_objs = os.listdir(cur_inst_folder)
    cur_inst_objs = [fn for fn in cur_inst_objs if fn.endswith(".obj")]
    for obj_fn in cur_inst_objs:
      obj_nm = obj_fn.split(".")[0]
      subd_nm, subd_idx = obj_nm.split("_")[0], obj_nm.split("_")[1]
      subd_idx = int(subd_idx)
      tot_obj_fn = os.path.join(cur_inst_folder, obj_fn)
      verts, faces = read_obj_file_ours(tot_obj_fn)
      nn_verts = verts.shape[0]
      flat_faces = flatten_faces(faces)
      nn_face_idxes = flat_faces.shape[0]
      cur_inst_info_dict[subd_idx] = (nn_verts, nn_face_idxes) #### inst_info_dict
    inst_fn_to_inst_info[cur_inst_folder] = cur_inst_info_dict
  cur_inst_meta_info_fn = os.path.join(dataset_root_path, "meta_infos.npy")
  np.save(cur_inst_meta_info_fn, inst_fn_to_inst_info)

# dataset_root_path = "/nas/datasets/gen/datasets/PartNet/data_v0_deci_0.2"
# get_partnet_meta_info(dataset_root_path)

########### motion dataset #############
# motion_dataset_root_path = "/nas/datasets/gen/datasets/MotionDataset_processed"
# motion_dataset_root_path = "/nas/datasets/gen/datasets/MotionDataset_processed_deci_part_first_0.2_normalized"
# motion_dataset_root_path = "/nas/datasets/gen/datasets/MotionDataset_processed_collapsed_sel"
# tot_motion_cats = os.listdir(motion_dataset_root_path)
# tot_motion_cats = [fn for fn in tot_motion_cats if os.path.isdir(os.path.join(motion_dataset_root_path, fn))]
# for motion_cat in tot_motion_cats:
#   cur_motion_cat_folder = os.path.join(motion_dataset_root_path, motion_cat)
#   get_motion_dataset_meta_info(cur_motion_cat_folder)
########### motion dataset #############

def touch_meta_info(meta_info_fn,):
  valid_bsp_obj_fns = [f"/{idxx}_bsp" for idxx in [0, 1, 3, 4, 6, 32]]
  new_meta_infos = {}
  meta_infos = np.load(meta_info_fn, allow_pickle=True).item()
  print(meta_infos)
  # for fnn in meta_infos:
  #   vvv = False
  #   for val_obj_fn in valid_bsp_obj_fns:
  #     if val_obj_fn in fnn:
  #       vvv = True
  #       break
  #   if vvv:
  #     new_meta_infos[fnn] = meta_infos[fnn]
  # print(new_meta_infos)
  # np.save(meta_info_fn, new_meta_infos)
  return new_meta_infos
    



motion_dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh/screw_1"
motion_dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh/door_1"
motion_dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh/screw_2"
motion_dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh/eyeglasses_1_merged"
motion_dataset_root_path = "/nas/datasets/gen/datasets/bsp_abstraction/eyeglasses_2"
motion_dataset_root_path = "/nas/datasets/gen/datasets/bsp_abstraction_2/eyeglasses"
motion_dataset_root_path = "/nas/datasets/gen/datasets/bsp_abstraction_4/eyeglasses"
motion_dataset_root_path = "/nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses"
motion_dataset_root_path = "/share/xueyi/datasets/gen/datasets/bsp_abstraction_5/scissors"
sub_folders = os.listdir(motion_dataset_root_path)
# sub_folders = ["merged_2"]
# sub_folders = ["merged_3"]
sub_folders = [fn for fn in sub_folders if os.path.isdir(os.path.join(motion_dataset_root_path, fn))]
for sub_folder in sub_folders:
  cur_sub_folder = os.path.join(motion_dataset_root_path, sub_folder)
  get_pure_foler_meta_info(cur_sub_folder)
# get_pure_foler_meta_info(motion_dataset_root_path)


# motion_dataset_root_path = "/nas/datasets/gen/datasets/bsp_abstraction_3/eyeglasses"
# subd_folder = "merged_2"
# meta_info_fn = "meta_infos.npy"
# # cur_meta_info_fn = "/nas/datasets/gen/datasets/bsp_abstraction_3/eyeglasses/merged_2/meta_infos.npy"
# cur_meta_info_fn = os.path.join(motion_dataset_root_path, subd_folder, meta_info_fn)
# touch_meta_info(cur_meta_info_fn)

# 0 1 3 4 6 32





# subd_dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh/lamp_1_reformed"
# subd_dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh/eyeglasses_1_reformed"
# subd_dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh/bsp_eyeglasses"
# subd_dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh/pen_1_reformed"
# subd_dataset_root_path = "/nas/datasets/gen/datasets/subd_mesh/bsp_eyeglasses_2"
# get_subd_dataset_meta_info(subd_dataset_root_path)

Meta-infos saved to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/cvx_3/meta_infos.npy!
Meta-infos saved to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/cvx_1/meta_infos.npy!
Meta-infos saved to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/cvx_2/meta_infos.npy!
Meta-infos saved to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/meta_infos.npy!


In [25]:

def reform_dataset(data_root_folder, sv_data_root_folder):
  all_subd_folders = os.listdir(data_root_folder)
  # all_subd_folders = [fn for fn in all_subd_folders if os.path.isdir(os.path.join(data_root_folder, fn))]
  all_subd_folders = ["subd0", "subd1", "subd2", "subd3", "subd4", "subd5"]
  all_inst_objs = os.listdir(os.path.join(data_root_folder, all_subd_folders[0]))
  all_inst_objs = [fn for fn in all_inst_objs if fn.endswith(".obj")]
  all_inst_idxes = [int(fn.split(".")[0]) for fn in all_inst_objs]
  for inst_idx in all_inst_idxes:
    cur_inst_sv_folder = os.path.join(sv_data_root_folder, str(inst_idx))
    os.makedirs(cur_inst_sv_folder, exist_ok=True)
    for subd_folder in all_subd_folders:
      cur_subd_idx = int(subd_folder[-1])
      cur_inst_cur_subd_obj_fn = os.path.join(data_root_folder, subd_folder, str(inst_idx) + ".obj")
      cur_inst_cur_subd_sv_obj_fn = os.path.join(cur_inst_sv_folder, f"subd_{cur_subd_idx}.obj")
      print(cur_subd_idx, inst_idx)
      os.system(f"cp {cur_inst_cur_subd_obj_fn} {cur_inst_cur_subd_sv_obj_fn}")


def reform_dataset_mult_inst(data_root_folder, sv_data_root_folder):
  # all_subd_folders = os.listdir(data_root_folder)
  # all_subd_folders = [fn for fn in all_subd_folders if os.path.isdir(os.path.join(data_root_folder, fn))]
  # all_subd_folders = ["subd0", "subd1", "subd2", "subd3", "subd4", "subd5"]

  subdn = 7
  all_subd_folders = [f"subd{i_subd}" for i_subd in range(subdn)]
# 
  motion_insts = os.listdir(data_root_folder)
  motion_insts = [fn for fn in motion_insts if os.path.isdir(os.path.join(data_root_folder, fn)) and fn[0] != "_"]



  inst_nn = 0
  for i_inst, cur_motion_inst in enumerate(motion_insts):
    print(f"Processing instance: {cur_motion_inst}")
    cur_motion_inst_folder = os.path.join(data_root_folder, cur_motion_inst)
    all_inst_objs = os.listdir(os.path.join(cur_motion_inst_folder, all_subd_folders[0]))
    all_inst_objs = [fn for fn in all_inst_objs if fn.endswith(".obj")]
    for cur_inst_obj in all_inst_objs:
      cur_inst_sv_folder = os.path.join(sv_data_root_folder, str(inst_nn))
      os.makedirs(cur_inst_sv_folder, exist_ok=True)
      inst_nn += 1
      for subd_folder in all_subd_folders:
        cur_subd_idx = int(subd_folder[-1])
        cur_inst_cur_subd_sv_fn = os.path.join(cur_inst_sv_folder, f"subd_{cur_subd_idx}.obj")
        cur_inst_cur_subd_fr_fn = os.path.join(cur_motion_inst_folder, subd_folder, cur_inst_obj)
        os.system(f"cp {cur_inst_cur_subd_fr_fn} {cur_inst_cur_subd_sv_fn}") #### save obj to the target position
        
  # all_inst_objs = os.listdir(os.path.join(data_root_folder, all_subd_folders[0]))
  # all_inst_objs = [fn for fn in all_inst_objs if fn.endswith(".obj")]
  # all_inst_idxes = [int(fn.split(".")[0]) for fn in all_inst_objs]
  # for inst_idx in all_inst_idxes:
  #   cur_inst_sv_folder = os.path.join(sv_data_root_folder, str(inst_idx))
  #   os.makedirs(cur_inst_sv_folder, exist_ok=True)
  #   for subd_folder in all_subd_folders:
  #     cur_subd_idx = int(subd_folder[-1])
  #     cur_inst_cur_subd_obj_fn = os.path.join(data_root_folder, subd_folder, str(inst_idx) + ".obj")
  #     cur_inst_cur_subd_sv_obj_fn = os.path.join(cur_inst_sv_folder, f"subd_{cur_subd_idx}.obj")
  #     print(cur_subd_idx, inst_idx)
  #     os.system(f"cp {cur_inst_cur_subd_obj_fn} {cur_inst_cur_subd_sv_obj_fn}")


root_folder = "/nas/datasets/gen/datasets/subd_mesh"
# os.makedirs(root_folder, exist_ok=True)

# category_folder = "bunny_4"
# category_folder = "bunny_6"
# category_folder = "pen_1"
# category_folder = "motion_4"
tot_cat_nms = ["bob_1", "cactus_1", "fish_1", "horse_1", "spot_1"]
tot_cat_nms = ["motion_5", ]
tot_cat_nms = ["screw_2", ]
tot_cat_nms = ["lamp_1", ]
tot_cat_nms = ["eyeglasses_1", ]
tot_cat_nms = ["pen_1", ]
for cat_nm in tot_cat_nms:
  # category_folder = "motion_4"
  category_folder = cat_nm
  data_root_folder = os.path.join(root_folder, category_folder)

  # sv_category_folder = "bunny_6_reformed"
  # sv_category_folder = "pen_1_reformed"
  sv_category_folder = f"{cat_nm}_reformed"
  sv_data_root_folder = os.path.join(root_folder, sv_category_folder)
  os.makedirs(sv_data_root_folder, exist_ok=True)

  # reform_dataset(data_root_folder, sv_data_root_folder)
  reform_dataset_mult_inst(data_root_folder, sv_data_root_folder)



Processing instance: 0030
Processing instance: 0001
Processing instance: 0046
Processing instance: 0041
Processing instance: 0038
Processing instance: 0031
Processing instance: 0032
Processing instance: 0022
Processing instance: 0043
Processing instance: 0002
Processing instance: 0023
Processing instance: 0048
Processing instance: 0033
Processing instance: 0020
Processing instance: 0003
Processing instance: 0008
Processing instance: 0025
Processing instance: 0028
Processing instance: 0049
Processing instance: 0037
Processing instance: 0052
Processing instance: 0044
Processing instance: 0051
Processing instance: 0035
Processing instance: 0013
Processing instance: 0034
Processing instance: 0006
Processing instance: 0021
Processing instance: 0026
Processing instance: 0050
Processing instance: 0024
Processing instance: 0045
Processing instance: 0039
Processing instance: 0009
Processing instance: 0015
Processing instance: 0005
Processing instance: 0004
Processing instance: 0014
Processing i

In [5]:
# 

# from asyncio.streams import _ClientConnectedCallback


def get_edges(faces):
  edge_to_exist = {}
  edge_list = []
  for i_f, cur_f in enumerate(faces):
    for i0, v0 in enumerate(cur_f):
      i1 = (i0 + 1) % len(cur_f)
      v1 = cur_f[i1]
      edge_pair = (v0, v1) if v0 < v1 else (v1, v0)
      if edge_pair not in edge_to_exist:
        edge_to_exist[edge_pair] = 1
        edge_list.append(edge_pair)
  return edge_list

def get_vertices_connected(vertices, faces):
  vert_idx = 0
  old_vert_idx_to_new_vert_idx = {}
  connected_vertices = []
  for i_f, cur_f in enumerate(faces):
    for cur_v in cur_f:
      if cur_v not in old_vert_idx_to_new_vert_idx:
        old_vert_idx_to_new_vert_idx[cur_v] = vert_idx + 1
        vert_idx += 1
        connected_vertices.append(cur_v - 1)
  print(f"connected_vertices: {len(connected_vertices)}")
  connected_vertices = np.array(connected_vertices, dtype=np.long)
  connected_vertices = vertices[connected_vertices]
  
  new_faces = []
  for i_f, cur_f in enumerate(faces):
    cur_new_f = [old_vert_idx_to_new_vert_idx[cur_v] for cur_v in cur_f]
    new_faces.append(cur_new_f)
  
  return connected_vertices, new_faces
  
 
def test_subd(obj_nm):
  vertices, faces = read_obj_file_ours(obj_nm)
  edge_list = get_edges(faces)
  
  print(f"vertices: {vertices.shape}, edge_list: {len(edge_list)}")

  vertices, faces = get_vertices_connected(vertices, faces)
  edge_list = get_edges(faces)
  print(f"vertices: {vertices.shape}, edge_list: {len(edge_list)}")


obj_nm = "/nas/datasets/gen/datasets/subd_mesh/bunny_5/49/subd_1.obj"
test_subd(obj_nm)


  

NameError: name 'read_obj_file_ours' is not defined

In [7]:

import data_utils

def get_subd_data_statistics(data_folder):
  subd_levels = os.listdir(data_folder)
  subd_levels = [fn for fn in subd_levels if os.path.isdir(os.path.join(data_folder, fn))]
  for subd_fn in subd_levels:
    cur_subd_folder = os.path.join(data_folder, subd_fn)
    cur_subd_obj_fns = os.listdir(cur_subd_folder)
    cur_subd_obj_fns = [fn for fn in cur_subd_obj_fns if fn.endswith(".obj")]
    obj_nn_verts_list = []
    obj_nn_faces_list = []
    for cur_obj_fn in cur_subd_obj_fns:
      cur_tot_obj_fn = os.path.join(cur_subd_folder, cur_obj_fn)
      verts, faces = data_utils.read_obj_file_ours(cur_tot_obj_fn) #### verts, faces
      cur_obj_nn_verts, cur_obj_nn_faces = verts.shape[0], len(faces)
      obj_nn_verts_list.append(cur_obj_nn_verts)
      obj_nn_faces_list.append(cur_obj_nn_faces)
    avg_obj_nn_verts = float(sum(obj_nn_verts_list)) / float(len(obj_nn_verts_list))
    avg_obj_nn_faces = float(sum(obj_nn_faces_list)) / float(len(obj_nn_faces_list))
    print(f"subd: {subd_fn}, avg_nn_verts: {avg_obj_nn_verts}, avg_nn_faces: {avg_obj_nn_faces}")

# data_folder = "/home/xueyi/gen/neuralSubdiv/data_meshes/cartoon_elephant_200"
# get_subd_data_statistics(data_folder)


def create_data_for_neural_subdiv(data_root_folder):
  # data_names = ["bob_1", ]
  # dst_data_names = ["bob_1_neural_subdiv"]
  data_names = ["cactus_1", "fish_1", "horse_1", "spot_1"]
  dst_data_names = [nm + "_neural_subdiv" for nm in data_names]
  for cur_data_nm, dst_data_nm in zip(data_names, dst_data_names):
    # cur_data_nm = os.path.join(data_root_folder, )
    tot_data_nm = os.path.join(data_root_folder, cur_data_nm)
    # os.makedirs(tot_data_nm, exist_ok=True)
    tot_dst_data_nm = os.path.join(data_root_folder, dst_data_nm) #### tot_dst_data_nm --> data_root_folder, dst_data_nm
    os.makedirs(tot_dst_data_nm, exist_ok=True)
    # src_subd_folders = ["subd2", "subd3", "subd4"]
    # dst_subd_folders = ["subd0", "subd1", "subd2"]

    src_subd_folders = os.listdir(tot_data_nm)
    src_subd_folders = [fn for fn in src_subd_folders if os.path.isdir(os.path.join(tot_data_nm, fn))]
    dst_subd_folders = src_subd_folders
    for src_folder, dst_folder in zip(src_subd_folders, dst_subd_folders):
      
      cur_dst_folder = os.path.join(tot_dst_data_nm, dst_folder)
      # cur_src_folder = os.path.join(tot_dst_data_nm, )
      cur_src_folder = os.path.join(tot_data_nm, src_folder)
      os.system(f"cp -r {cur_src_folder} {cur_dst_folder}")
    # print(f)
data_root_folder = "/nas/datasets/gen/datasets/subd_mesh"
create_data_for_neural_subdiv(data_root_folder)

In [4]:

import pymeshlab
from data_utils import read_obj_file_ours

def process_mesh_edge_collapse(data_root_path, sv_data_root_path):
  print(f"Simplifying meshes via edge collapsing from root_path: {data_root_path} to {sv_data_root_path}...")
  motion_cats = ["eyeglasses"]
  part_nms = ["none_motion"]
  os.makedirs(sv_data_root_path, exist_ok=True)
  for cur_motion_cat in motion_cats:
    print(f"Processing category {cur_motion_cat}...")
    fr_motion_root_path = os.path.join(data_root_path, cur_motion_cat)
    sv_motion_root_path = os.path.join(sv_data_root_path, cur_motion_cat)
    os.makedirs(sv_motion_root_path, exist_ok=True)
    
    inst_fns = os.listdir(fr_motion_root_path)
    inst_fns = [fn for fn in inst_fns if os.path.isdir(os.path.join(fr_motion_root_path, fn))]
    for cur_inst_fn in inst_fns:
      print(f" Processing instnace {cur_inst_fn}...")
      fr_inst_root_path = os.path.join(fr_motion_root_path, cur_inst_fn)
      sv_inst_root_path = os.path.join(sv_motion_root_path, cur_inst_fn)
      os.makedirs(sv_inst_root_path, exist_ok=True)
      for cur_part_nm in part_nms:
        cur_fr_obj_fn = os.path.join(fr_inst_root_path, cur_part_nm + ".obj")
        # collapse_n_faces = range(30, 200, 10)
        collapse_n_faces = [30, 40, 50]
        verts, faces = read_obj_file_ours(cur_fr_obj_fn)
        cur_n_faces = len(faces)
        for collapse_n_face in collapse_n_faces:
          if collapse_n_face >= cur_n_faces:
            break
          ms = pymeshlab.MeshSet()
          ms.load_new_mesh(cur_fr_obj_fn)
          ms.apply_filter('simplification_quadric_edge_collapse_decimation', targetfacenum=collapse_n_face,)
          cur_collapsed_sv_fn = cur_part_nm + f"_collapse_n_{collapse_n_face}" + ".obj"
          cur_collapsed_sv_fn = os.path.join(sv_inst_root_path, cur_collapsed_sv_fn)
          ms.save_current_mesh(cur_collapsed_sv_fn)



data_root_path = "/nas/datasets/gen/datasets/MotionDataset_processed"
sv_data_root_path = "/nas/datasets/gen/datasets/MotionDataset_processed_collapsed_sel"

process_mesh_edge_collapse(data_root_path, sv_data_root_path)



Simplifying meshes via edge collapsing from root_path: /nas/datasets/gen/datasets/MotionDataset_processed to /nas/datasets/gen/datasets/MotionDataset_processed_collapsed_sel...
Processing category eyeglasses...
 Processing instnace 0030...
 Processing instnace 0001...
 Processing instnace 0041...
 Processing instnace 0038...
 Processing instnace 0031...
 Processing instnace 0032...
 Processing instnace 0022...
 Processing instnace 0043...
 Processing instnace 0002...
 Processing instnace 0023...
 Processing instnace 0033...
 Processing instnace 0020...
 Processing instnace 0003...
 Processing instnace 0008...
 Processing instnace 0025...
 Processing instnace 0028...
 Processing instnace 0037...
 Processing instnace 0035...
 Processing instnace 0013...
 Processing instnace 0034...
 Processing instnace 0006...
 Processing instnace 0021...
 Processing instnace 0026...
 Processing instnace 0024...
 Processing instnace 0039...
 Processing instnace 0009...
 Processing instnace 0015...
 Proce

In [9]:

import os

def reform_to_subd_folder(fr_folder, sv_folder, subdn=5):
  os.makedirs(sv_folder, exist_ok=True)
  inst_objs = os.listdir(fr_folder)
  inst_objs = [fn for fn in inst_objs if fn.endswith(".obj")]
  for i, cur_inst_obj_fn in enumerate(inst_objs):
    cur_inst_fr_obj_fn = os.path.join(fr_folder, cur_inst_obj_fn)
    cur_inst_folder = os.path.join(sv_folder, str(i))
    os.makedirs(cur_inst_folder, exist_ok=True)
    for i_subd in range(subdn):
      cur_subd_obj_fn = f"subd_{i_subd}.obj"
      cur_subd_obj_fn = os.path.join(cur_inst_folder, cur_subd_obj_fn)
      os.system(f"cp {cur_inst_fr_obj_fn} {cur_subd_obj_fn}")

samples_root_folder = "/nas/datasets/gen/samples"
fr_folder = "sampling_test_scale_simple_beam_False_ar_part_indi_True_screw_2_cat_eyeglasses_bsz_1_max_verts_600_faces_3000_max_permit_verts_600_faces_3000_mask_low_0.15_high_0.16"
fr_folder = os.path.join(samples_root_folder, fr_folder)
sv_folder = fr_folder + "_subd"
sv_folder = os.path.join(samples_root_folder, sv_folder)
reform_to_subd_folder(fr_folder, sv_folder, subdn=5)


In [20]:


def merge_eyeglasses_data(fr_folder, sv_folder, cur_subd=0):
  insts = os.listdir(fr_folder)
  insts = [fn for fn in insts if os.path.isdir(os.path.join(fr_folder, fn))]
  subd_nm = f"subd{cur_subd}"
  merge_folder = f"subd{cur_subd}"
  merge_folder = os.path.join(sv_folder, merge_folder)
  os.makedirs(sv_folder, exist_ok=True)
  os.makedirs(merge_folder, exist_ok=True)
  tot_n = 0
  for cur_inst in insts:
    cur_inst_folder = os.path.join(fr_folder, cur_inst)
    cur_inst_subd_folder = os.path.join(cur_inst_folder, subd_nm)
    cur_inst_subd_objs = os.listdir(cur_inst_subd_folder)
    cur_inst_subd_objs = [fn for fn in cur_inst_subd_objs if fn.endswith(".obj")]
    for cur_inst_obj in cur_inst_subd_objs:
      cur_inst_obj_fn = os.path.join(cur_inst_subd_folder, cur_inst_obj)
      verts, faces = read_obj_file_ours(cur_inst_obj_fn)
      if verts.shape[0] > 0 and len(faces) > 0:
        cur_sv_obj_fn = os.path.join(merge_folder, f"{tot_n}.obj")
        save_obj_file(verts, faces, cur_sv_obj_fn)
        tot_n += 1
  print(f"tot_n: {tot_n}")

fr_folder = "/nas/datasets/gen/datasets/subd_mesh/eyeglasses_1"
sv_folder = "/nas/datasets/gen/datasets/subd_mesh/eyeglasses_1_merged"
merge_eyeglasses_data(fr_folder, sv_folder, 0)

tot_n: 1100


In [3]:


def merge_bsp_data(rt_folder, cvx_list=[35, 168, 175], merged_flag="merged_1", merge_data_idx=[0,1]):
  if cvx_list is None:
    tot_cvx_list = os.listdir(rt_folder)
    tot_cvx_list = [fn for fn in tot_cvx_list if os.path.isdir(os.path.join(rt_folder, fn)) and "cvx" in fn]
    cvx_list = [int(idxx.split("_")[1]) for idxx in tot_cvx_list]
  print(f"cvx list: {cvx_list}")
  merged_data_fn = os.path.join(rt_folder, merged_flag)
  os.makedirs(merged_data_fn, exist_ok=True)
  for idx in merge_data_idx:
    verts_list = []
    faces_list = []
    print(f"idx: {idx}")
    for cvx in cvx_list:
      cur_cvx_folder = os.path.join(rt_folder, f"cvx_{cvx}")
      cur_idx_cur_cvx_obj_fn = os.path.join(cur_cvx_folder, f"{idx}_bsp.obj")
      if not os.path.exists(cur_idx_cur_cvx_obj_fn):
        continue
      cur_verts, cur_faces = read_obj_file_ours(cur_idx_cur_cvx_obj_fn)
      verts_list.append(cur_verts)
      faces_list.append(cur_faces)
    if len(verts_list) > 0 and len(faces_list) > 0:
      
      verts, faces = merge_meshes(verts_list, faces_list)
      cur_idx_sv_fn = os.path.join(merged_data_fn, f"{idx}_bsp.obj")
      print(f"Saving to {cur_idx_sv_fn}")
      save_obj_file(verts, faces, cur_idx_sv_fn)

rt_folder = "/nas/datasets/gen/datasets/bsp_abstraction_2/eyeglasses"
rt_folder = "/nas/datasets/gen/datasets/bsp_abstraction_3/eyeglasses"
rt_folder = "/nas/datasets/gen/datasets/bsp_abstraction_4/eyeglasses"
rt_folder = "/nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses"
merged_flag = "merged_1"
# merged_flag = "merged_2"

# merge_data_idx=[0,1]
merge_data_idx = range(43)
# cvx_list=[35, 168, 175]
# cvx_list=[113, 168, 212]
cvx_list=None
merge_bsp_data(rt_folder, cvx_list, merged_flag, merge_data_idx)
  


cvx list: [3, 1, 2]
idx: 0
Saving to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/0_bsp.obj
idx: 1
Saving to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/1_bsp.obj
idx: 2
Saving to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/2_bsp.obj
idx: 3
Saving to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/3_bsp.obj
idx: 4
Saving to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/4_bsp.obj
idx: 5
Saving to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/5_bsp.obj
idx: 6
Saving to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/6_bsp.obj
idx: 7
Saving to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/7_bsp.obj
idx: 8
Saving to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/8_bsp.obj
idx: 9
Saving to /nas/datasets/gen/datasets/bsp_abstraction_5/eyeglasses/merged_1/9_bsp.obj
idx: 10
Saving to /nas/datasets/gen/datasets/bsp_abstraction

In [ ]:

import h5py

def test_gathered_data(gathered_data_root):
  tot_cats = os.listdir(gathered_data_root)
  tot_cats = [fn for fn in tot_cats if os.path.isdir(os.path.join(gathered_data_root, fn))]
  for cat in tot_cats:
    cur_cat_hdf5_fn = os.path.join(gathered_data_root, cat, "models_train.hdf5")
    try:
      data_dict = h5py.File(cur_cat_hdf5_fn, 'r')
    except:
      print(f"{cur_cat_hdf5_fn}")  